### Scraping the link

In [ ]:
import undetected_chromedriver as uc
import random

def random_user_agent():
    # This can be expanded. Use a library or list to randomly select a user-agent.
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36",
        # ... add more user-agents
    ]
    return random.choice(user_agents)

def random_delay(minimum=5, maximum=15):
    return time.sleep(random.uniform(minimum, maximum))

options = uc.ChromeOptions()
options.add_argument(f"user-agent={random_user_agent()}")

# Add other options like proxies here, if you have them

browser = uc.Chrome(options=options)

base_url = "https://www.realtor.com/apartments/Las-Vegas_NV/pg-10"


property_links = []

browser.get(url)

    # Introduce random delay after loading each page
random_delay()

    # Scroll through the page to load more content
scroll_page()

    # Wait until the presence of the property links
wait = WebDriverWait(browser, 60)  # Increased the wait time to 60 seconds
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.card-anchor")))

    # Extract the property links from the page
links = browser.find_elements(By.CSS_SELECTOR, "a.card-anchor")
for link in links:
    property_links.append(link.get_attribute("href"))

    # Introduce random delay before moving to the next page
random_delay()

# Close the browser
browser.quit()

# Save links to CSV
with open('property_links.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Link'])
    for link in property_links:
        writer.writerow([link])

print("Scraping completed and saved to property_links.csv!")


### Scraping each links information

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re

# List of links you want to scrape
links = [
'https://www.realtor.com/rentals/details/3501-S-Maryland-Pkwy_Las-Vegas_NV_89169_M14058-58461',
'https://www.realtor.com/rentals/details/4000-E-Bonanza-Rd_Las-Vegas_NV_89110_M27725-71219',
'https://www.realtor.com/rentals/details/2120-Van-Patten-Pl_Las-Vegas_NV_89104_M19992-31808',
'https://www.realtor.com/rentals/details/9490-Bermuda-Rd_Las-Vegas_NV_89123_M14284-51408',
'https://www.realtor.com/rentals/details/3896-University-Center-Dr_Las-Vegas_NV_89119_M95695-89840',
'https://www.realtor.com/rentals/details/2255-Spruce-Goose-St_Las-Vegas_NV_89135_M97751-35398',
'https://www.realtor.com/rentals/details/9950-W-Tropicana-Ave_Las-Vegas_NV_89147_M97868-55464',
'https://www.realtor.com/rentals/details/5091-Moose-Falls-Dr_Las-Vegas_NV_89141_M29539-47285',
'https://www.realtor.com/rentals/details/6978-Walden-Park-St_Las-Vegas_NV_89166_M17335-05214',
'https://www.realtor.com/rentals/details/7565-Norman-Rockwell-Ln-Ste-110_Las-Vegas_NV_89143_M95950-01113',
'https://www.realtor.com/rentals/details/11055-Las-Vegas-Blvd-S_Las-Vegas_NV_89183_M90939-36612',
'https://www.realtor.com/rentals/details/7659-Glorietta-Bay-Ct_Las-Vegas_NV_89139_M19296-60665',
'https://www.realtor.com/rentals/details/3400-Cabana-Dr-15_Las-Vegas_NV_89122_M94543-50448',
'https://www.realtor.com/rentals/details/3024-Reef-View-St_Las-Vegas_NV_89117_M25843-96157',
'https://www.realtor.com/rentals/details/9156-Conquest-Ct_Las-Vegas_NV_89149_M22396-41923',
'https://www.realtor.com/rentals/details/5164-Mineral-Lake-Dr_Las-Vegas_NV_89122_M21178-23409',
'https://www.realtor.com/rentals/details/3750-Corpolo-Ave_Las-Vegas_NV_89141_M23958-51519',
'https://www.realtor.com/rentals/details/7209-Beachwood-Crest-St_Las-Vegas_NV_89166_M25908-79728',
'https://www.realtor.com/rentals/details/9428-Fort-Connah-Ct_Las-Vegas_NV_89178_M28705-83371',
'https://www.realtor.com/rentals/details/10617-Capitol-Peak-Ave_Las-Vegas_NV_89166_M10535-47525',
'https://www.realtor.com/rentals/details/8262-Vanity-Fair-Ln_Las-Vegas_NV_89113_M22681-13345',
'https://www.realtor.com/rentals/details/4715-Longshot-Dr_Las-Vegas_NV_89122_M17389-80283',
'https://www.realtor.com/rentals/details/7941-Violet-Dawn-St_Las-Vegas_NV_89149_M26442-87048',
'https://www.realtor.com/rentals/details/7105-Powderhorn-Cir_Las-Vegas_NV_89128_M28344-00770',
'https://www.realtor.com/rentals/details/6460-Hamilton-Grove-Ave_Las-Vegas_NV_89122_M24350-90418',
'https://www.realtor.com/rentals/details/5033-Whistling-Acres-Ave_Las-Vegas_NV_89131_M11631-14748',
'https://www.realtor.com/rentals/details/7912-Point-Oaks-Ct_Las-Vegas_NV_89149_M24793-02712',
'https://www.realtor.com/rentals/details/4792-W-Shelbourne-Ave_Las-Vegas_NV_89139_M13639-06795',
'https://www.realtor.com/rentals/details/7112-Mercurio-Ave_Las-Vegas_NV_89131_M25641-73407',
'https://www.realtor.com/rentals/details/7315-Larix-Rd_Las-Vegas_NV_89113_M29691-85907',
'https://www.realtor.com/rentals/details/8261-Oliver-Twist-Ln_Las-Vegas_NV_89113_M10510-50707',
'https://www.realtor.com/rentals/details/10525-Autumn-Pine-Ave-Apt-205_Las-Vegas_NV_89144_M13423-92454',
'https://www.realtor.com/rentals/details/8819-W-Oquendo-Rd_Las-Vegas_NV_89148_M10494-27730',
'https://www.realtor.com/rentals/details/595-S-Royal-Crest-Cir-Unit-7_Las-Vegas_NV_89169_M17649-29455',
'https://www.realtor.com/rentals/details/9594-Vega-Carpio-Ave_Las-Vegas_NV_89178_M28291-64246',
'https://www.realtor.com/rentals/details/5894-Grechetto-Ct_Las-Vegas_NV_89141_M26091-82381',
'https://www.realtor.com/rentals/details/8920-Picket-Fence-Ave_Las-Vegas_NV_89143_M29292-10403',
'https://www.realtor.com/rentals/details/4973-Blue-Marlin-Ave_Las-Vegas_NV_89115_M25339-22847',
'https://www.realtor.com/rentals/details/5725-Smoke-Ranch-Rd-Unit-C_Las-Vegas_NV_89108_M16486-35746',
'https://www.realtor.com/rentals/details/7224-Cottonsparrow-St_Las-Vegas_NV_89131_M15670-54301',
'https://www.realtor.com/rentals/details/725-S-Hualapai-Way_Las-Vegas_NV_89145_M10327-15621',
'https://www.realtor.com/rentals/details/3240-Las-Vegas-Blvd-N_Las-Vegas_NV_89115_M15790-49379',
'https://www.realtor.com/rentals/details/2700-N-Rainbow-Blvd_Las-Vegas_NV_89108_M12346-64196',
'https://www.realtor.com/rentals/details/9625-W-Russell-Rd_Las-Vegas_NV_89148_M23144-53806',
'https://www.realtor.com/rentals/details/818-E-Flamingo-Rd_Las-Vegas_NV_89119_M16729-90948',
'https://www.realtor.com/rentals/details/2375-Spruce-Goose-St_Las-Vegas_NV_89135_M23574-93064',
'https://www.realtor.com/rentals/details/6614-Blue-Diamond-Rd_Las-Vegas_NV_89139_M98789-01931',
'https://www.realtor.com/rentals/details/10136-Serenity-Star-Way_Las-Vegas_NV_89147_M94416-26958',
'https://www.realtor.com/rentals/details/9501-W-Sahara-Ave_Las-Vegas_NV_89117_M10102-07234',
'https://www.realtor.com/rentals/details/4450-Karen-Ave_Las-Vegas_NV_89121_M29403-05227',

]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

data_to_save = []

for link in links:
    response = requests.get(link, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage: {link}")
        continue

    # Parse page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract apartment name
    name_tag = soup.find('h1', {'class': lambda x: x and 'base__StyledType-rui' in x and 'name' in x})
    apartment_name = name_tag.text if name_tag else None
    
    # Extract apartment address
    address_tag = soup.find('p', {'class': lambda x: x and 'base__StyledType-rui' in x and 'address' in x})
    apartment_address = address_tag.text.strip() if address_tag else None

    # Extract ZIP code
    zip_code = None
    if apartment_address:
        zip_code = apartment_address.split()[-1]
    
    # Extract price
    price_tag = soup.find('div', class_=re.compile('Pricestyles__StyledPrice-rui__'))
    price = price_tag.text.split('-')[0].strip().replace('$', '') if price_tag else None

    # Extract square feet
    sqft_tag = soup.find('li', class_=re.compile('PropertySqftMetastyles__StyledPropertySqftMeta-rui__'))
    sqft = sqft_tag.find('span', class_='meta-value').text.split('-')[0].strip().replace('sqft', '').strip() if sqft_tag and sqft_tag.find('span', class_='meta-value') else None

    # Extract beds
    beds_tag = soup.find('li', attrs={"data-testid": "property-meta-beds"})
    beds_value = beds_tag.find('span', attrs={"data-testid": "meta-value"}).text if beds_tag else None

    if beds_value:
        if 'Studio' in beds_value:
            beds = 'Studio'
        else:
            beds = beds_value.split('-')[0].strip()

    # Extract baths
    baths_tag = soup.find('li', attrs={"data-testid": "property-meta-baths"})
    baths_value = baths_tag.find('span', attrs={"data-testid": "meta-value"}).text if baths_tag else None
    baths = baths_value.split('-')[0].strip() if baths_value else None
    
    # Extract Property Type
    property_type = None
    type_tag = soup.find('div', string='Property type')
    if type_tag and type_tag.find_next_sibling():
        property_type = type_tag.find_next_sibling().text

    # Extract Year Built
    year_built = None
    year_built_tag = soup.find('div', string='Year built')
    if year_built_tag and year_built_tag.find_next_sibling():
        year_built = year_built_tag.find_next_sibling().text
    
    # Append data to the list
    data_to_save.append([link, apartment_name, apartment_address, zip_code, price, sqft, beds, baths, property_type, year_built])


# Write data to CSV
with open('apartment_data56.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # Added 'URL' to the header
    writer.writerow(['URL', 'Apartment Name', 'Apartment Address', 'ZIP Code', 'Price', 'Sqft', 'Beds', 'Baths', 'Property Type', 'Year Built'])  
    writer.writerows(data_to_save)

print("Data saved to 'apartment_data.csv'")


Scrape Links



In [2]:
import csv
import requests
from bs4 import BeautifulSoup

def is_listing_link(tag):
    """Check if a tag is an anchor tag with a listing link."""
    return (tag.name == 'a' and 
            tag.has_attr('title') and 
            tag.has_attr('href') and
            'NV' in tag['title'])

def get_links(url):
    response = requests.get(url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.content, 'html.parser')
    listing_links = soup.find_all(is_listing_link)
    
    # Prepend the base URL to each link
    base_url = 'https://www.redfin.com'
    links = [f'{base_url}{link["href"]}' for link in listing_links]
    return links

def save_to_csv(data, filename):
    with open(filename, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for link in data:
            writer.writerow([link])

def main(urls, filename):
    for url in urls:
        print(f'Processing {url}...')
        links = get_links(url)
        save_to_csv(links, filename)
        print(f'Collected {len(links)} links from {url}')

# Example usage:
urls = [
   'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.31566:36.10735:-114.91856:-115.31441/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.31566:36.10735:-114.91856:-115.31441/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.31566:36.10735:-114.91856:-115.31441/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.31566:36.10735:-114.91856:-115.31441/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.22587:36.12166:-114.95273:-115.15066',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.22587:36.12166:-114.95273:-115.15066/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.22587:36.12166:-114.95273:-115.15066/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.22587:36.12166:-114.95273:-115.15066/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.22587:36.12166:-114.95273:-115.15066/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.22587:36.12166:-114.95273:-115.15066/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.22587:36.12166:-114.95273:-115.15066/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.17377:36.06949:-115.14481:-115.34274',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.17377:36.06949:-115.14481:-115.34274/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.17377:36.06949:-115.14481:-115.34274/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.17377:36.06949:-115.14481:-115.34274/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.17377:36.06949:-115.14481:-115.34274/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.17377:36.06949:-115.14481:-115.34274/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.17377:36.06949:-115.14481:-115.34274/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.17377:36.06949:-115.14481:-115.34274/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.17377:36.06949:-115.14481:-115.34274/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.21629:36.00772:-114.94777:-115.34362',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.21629:36.00772:-114.94777:-115.34362/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.21629:36.00772:-114.94777:-115.34362/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.21629:36.00772:-114.94777:-115.34362/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.21629:36.00772:-114.94777:-115.34362/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.21629:36.00772:-114.94777:-115.34362/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.21629:36.00772:-114.94777:-115.34362/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.21629:36.00772:-114.94777:-115.34362/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.21629:36.00772:-114.94777:-115.34362/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.14691:35.93815:-115.08255:-115.4784',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.14691:35.93815:-115.08255:-115.4784/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.14691:35.93815:-115.08255:-115.4784/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.14691:35.93815:-115.08255:-115.4784/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.14691:35.93815:-115.08255:-115.4784/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.14691:35.93815:-115.08255:-115.4784/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.14691:35.93815:-115.08255:-115.4784/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.14691:35.93815:-115.08255:-115.4784/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.14691:35.93815:-115.08255:-115.4784/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.07688:35.97248:-115.07801:-115.27593,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.07688:35.97248:-115.07801:-115.27593,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.07688:35.97248:-115.07801:-115.27593,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.07688:35.97248:-115.07801:-115.27593,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.07688:35.97248:-115.07801:-115.27593,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.07688:35.97248:-115.07801:-115.27593,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.07688:35.97248:-115.07801:-115.27593,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.07688:35.97248:-115.07801:-115.27593,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.07688:35.97248:-115.07801:-115.27593,no-outline/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.15807:35.94934:-114.76268:-115.15853,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.15807:35.94934:-114.76268:-115.15853,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.15807:35.94934:-114.76268:-115.15853,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.15807:35.94934:-114.76268:-115.15853,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.15807:35.94934:-114.76268:-115.15853,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.15807:35.94934:-114.76268:-115.15853,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.15807:35.94934:-114.76268:-115.15853,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.15807:35.94934:-114.76268:-115.15853,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.15807:35.94934:-114.76268:-115.15853,no-outline/page-9',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.29128:36.18717:-115.02848:-115.2264,no-outline',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.29128:36.18717:-115.02848:-115.2264,no-outline/page-2',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.29128:36.18717:-115.02848:-115.2264,no-outline/page-3',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.29128:36.18717:-115.02848:-115.2264,no-outline/page-4',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.29128:36.18717:-115.02848:-115.2264,no-outline/page-5',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.29128:36.18717:-115.02848:-115.2264,no-outline/page-6',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.29128:36.18717:-115.02848:-115.2264,no-outline/page-7',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.29128:36.18717:-115.02848:-115.2264,no-outline/page-8',
'https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.29128:36.18717:-115.02848:-115.2264,no-outline/page-9',

]
filename = 'Scraped_Links1.csv'

# Make sure to clear the old CSV file if it exists
with open(filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Link"])  # header

if __name__ == '__main__':
    main(urls, filename)


Processing https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.31566:36.10735:-114.91856:-115.31441/page-6...
Collected 40 links from https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.31566:36.10735:-114.91856:-115.31441/page-6
Processing https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.31566:36.10735:-114.91856:-115.31441/page-7...
Collected 40 links from https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.31566:36.10735:-114.91856:-115.31441/page-7
Processing https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.31566:36.10735:-114.91856:-115.31441/page-8...
Collected 40 links from https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.31566:36.10735:-114.91856:-115.31441/page-8
Processing https://www.redfin.com/city/10201/NV/Las-Vegas/filter/include=sold-1mo,viewport=36.31566:36.10735:-114.91856:-115

Scraping Features


In [3]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import re

def find_text(parent, text):
    element = parent.find(text=re.compile(text))
    return element.find_next('span').text if element else None

def extract_scores_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # This will raise an error if the GET request failed
    return extract_scores(response.content)

def extract_scores(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Function to extract score from div with given class_name
    def get_score(class_name):
        div = soup.find('div', class_=class_name)
        if div:
            percentage = div.find('span', {'class': 'value'})
            if percentage:
                return percentage.text
        return None  # Return None if score isn't found
    
    walk_score = get_score('walkscore')
    transit_score = get_score('transitscore')
    bike_score = get_score('bikescore')
    
    return walk_score, transit_score, bike_score

def scrape_school_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    school_data = []
    
    # Find all the school entries
    school_entries = soup.find_all("div", class_="ListItem col-10 clickable")
    
    for entry in school_entries:
        school_name_element = entry.find("h3", class_="ListItem__heading font-body-base-bold color-text-primary")
        school_score_element = entry.find("span", class_="rating-num font-size-base font-weight-bold")
        school_details_element = entry.find("p", class_="ListItem__description font-body-small-compact color-text-secondary")

        # If any element is not found, skip this entry
        if not school_name_element or not school_score_element or not school_details_element:
            continue

        school_name = school_name_element.text
        school_score = school_score_element.text
        school_details = school_details_element.text

        # Identify school type by checking the grades it serves
        if "PreK-5" in school_details:
            school_type = "Elementary School"
        elif "6-8" in school_details:
            school_type = "Middle School"
        elif "9-12" in school_details:
            school_type = "High School"
        else:
            school_type = "Unknown"
        
        school_data.append({
            "Name": school_name,
            "Type": school_type,
            "Score": school_score
        })

    return school_data
def extract_additional_details(soup):
    details = soup.find('div', class_='keyDetailsList')
    if not details:
        return None, None, None, None, None, None

    extracted = {}
    for row in details.find_all('div', class_='table-row'):
        label_elem = row.find('span', class_='table-label')
        value_elem = row.find('div', class_='table-value')

        if label_elem and value_elem:
            label = label_elem.text.strip()
            value = value_elem.text.strip()
            extracted[label] = value

    time_on_redfin = extracted.get('Time on Redfin', None)
    year_built = extracted.get('Year Built', None)
    property_type = extracted.get('Property Type', None)
    style = extracted.get('Style', None)
    hoa_dues = extracted.get('HOA Dues', None)
    lot_size = extracted.get('Lot Size', None)
    
    return time_on_redfin, year_built, property_type, style, hoa_dues, lot_size

def extract_additional_features(soup):
    
    def find_text(parent, text):
        element = parent.find(text=re.compile(text))
        return element.find_next('span').text if element else None
    
    
    exterior_features_div = soup.find('div', {'class': 'amenity-group'}, text=re.compile('Exterior Features'))

    garage_spaces = find_text(soup, '# of Garage Spaces:') or find_text(soup, '# of Carport Spaces:')
    floor_type = find_text(soup, 'Flooring:')
    heat_type = find_text(soup, 'Heating:')
    cool_type = find_text(soup, 'Cooling:')
    tax_amount = find_text(soup, 'Annual Amount:')
    
    
    patio = find_text(soup, 'Patio And Porch Features:')
    # Locate the 'li' element containing 'Pool Features:'
    pool_private = find_text(soup, 'Pool Features:')
    
        
    return garage_spaces, pool_private, floor_type, heat_type, cool_type, tax_amount, patio  # updated 'pool' to 'pool_private'
   



def extract_real_estate_info(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    top_stats = soup.find('div', class_='top-stats')
    if not top_stats:
        return None, None, None, None, None, None
    
    address_div = top_stats.find('header', class_='address')
    price_div = top_stats.find('div', {'class': 'stat-block', 'data-rf-test-id': 'abp-price'})
    beds_div = top_stats.find('div', {'class': 'stat-block', 'data-rf-test-id': 'abp-beds'})
    baths_div = top_stats.find('div', {'class': 'stat-block', 'data-rf-test-id': 'abp-baths'})
    sqft_div = top_stats.find('div', {'class': 'stat-block', 'data-rf-test-id': 'abp-sqFt'})
    est_payment_div = price_div.find('span', class_='est-monthly-payment') if price_div else None


    address = address_div.get_text(strip=True) if address_div else None
    price = price_div.div.text if price_div and price_div.div else None
    beds = beds_div.div.text if beds_div and beds_div.div else None
    baths = baths_div.div.text if baths_div and baths_div.div else None
    sqft = sqft_div.span.text if sqft_div and sqft_div.span else None
    est_payment = est_payment_div.text if est_payment_div else None
    time_on_redfin, year_built, property_type, style, hoa_dues, lot_size = extract_additional_details(soup)
    
    return address, price, beds, baths, sqft, est_payment, time_on_redfin, year_built, property_type, style, hoa_dues, lot_size



def combine_data_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # This will raise an error if the GET request failed
    soup = BeautifulSoup(response.content, 'html.parser')
    walk, transit, bike = extract_scores(response.content)
    school_data = scrape_school_data(url)
    address, price, beds, baths, sqft, est_payment, time_on_redfin, year_built, property_type, style, hoa_dues, lot_size = extract_real_estate_info(response.content)
    garage_spaces, pool_private, floor_type, heat_type, cool_type, tax_amount, patio = extract_additional_features(soup)


    elementary_school_name, elementary_school_score = None, None
    middle_school_name, middle_school_score = None, None
    high_school_name, high_school_score = None, None

    for school in school_data:
        if school["Type"] == "Elementary School" and not elementary_school_name:
            elementary_school_name = school["Name"]
            elementary_school_score = school["Score"]
        elif school["Type"] == "Middle School" and not middle_school_name:
            middle_school_name = school["Name"]
            middle_school_score = school["Score"]
        elif school["Type"] == "High School" and not high_school_name:
            high_school_name = school["Name"]
            high_school_score = school["Score"]
    
    return {
        "URL": url,
        "Walk Score (out of 100)": walk,
        "Transit Score (out of 100)": transit,
        "Bike Score (out of 100)": bike,
        "Elementary School Name": elementary_school_name,
        "Elementary School Score (out of 10)": elementary_school_score,
        "Middle School Name": middle_school_name,
        "Middle School Score (out of 10)": middle_school_score,
        "High School Name": high_school_name,
        "High School Score (out of 10)": high_school_score,
        "Address": address,
        "Price": price,
        "Beds": beds,
        "Baths": baths,
        "Sq Ft": sqft,
        "Est. Monthly Payment": est_payment,
        "Time on Redfin": time_on_redfin,
        "Year Built": year_built,
        "Property Type": property_type,
        "Style": style,
        "HOA Dues": hoa_dues,
        "Lot Size": lot_size,
        "Garage Spaces": garage_spaces,
        "Has Pool": pool_private,
        "Floor Type": floor_type,
        "Heat Type": heat_type,
        "Cool Type": cool_type,
        "Tax Amount": tax_amount,
        "Has Patio": patio,
       
    }


# Extract combined data for all URLs
urls = ['https://www.redfin.com/NV/Las-Vegas/3305-Grey-Dolphin-Dr-89117/home/29533177',
'https://www.redfin.com/NV/Las-Vegas/2750-S-Durango-Dr-89117/unit-1155/home/29535177',
'https://www.redfin.com/NV/Las-Vegas/8273-Ruby-Heights-Ave-89117/home/29535935',
'https://www.redfin.com/NV/Las-Vegas/7889-Mosaic-Harbor-Ave-89117/home/29536148',
'https://www.redfin.com/NV/Las-Vegas/8042-Earl-Grey-Ct-89117/home/29536662',
'https://www.redfin.com/NV/Las-Vegas/7225-Latour-Ct-89117/home/29538072',
'https://www.redfin.com/NV/Las-Vegas/6878-Edna-Ave-89146/home/29538264',
'https://www.redfin.com/NV/Las-Vegas/3150-S-Bronco-St-89146/home/29538548',
'https://www.redfin.com/NV/Las-Vegas/3858-Moongate-Cir-89103/home/29540882',
'https://www.redfin.com/NV/Las-Vegas/3797-Topawa-St-89103/home/29542434',
'https://www.redfin.com/NV/Las-Vegas/5280-Cabrito-Dr-89103/home/29542598',
'https://www.redfin.com/NV/Las-Vegas/6783-Baile-Rd-89146/home/29543729',
'https://www.redfin.com/NV/Las-Vegas/6576-Casa-Linda-Dr-89103/home/29543930',
'https://www.redfin.com/NV/Las-Vegas/6476-Scotch-Pine-Cir-89146/home/29544986',
'https://www.redfin.com/NV/Las-Vegas/6116-Meadowhaven-Ln-89103/home/29545621',
'https://www.redfin.com/NV/Las-Vegas/7400-W-Flamingo-Rd-89147/unit-2089/home/29546745',
'https://www.redfin.com/NV/Las-Vegas/3869-Bombastic-Ct-89147/home/29550220',
'https://www.redfin.com/NV/Las-Vegas/3519-Judah-Way-89147/home/29552180',
'https://www.redfin.com/NV/Las-Vegas/10015-Cambridge-Blue-Ave-89147/home/29555165',
'https://www.redfin.com/NV/Las-Vegas/9618-Windborne-Ave-89147/home/29555769',
'https://www.redfin.com/NV/Las-Vegas/3942-Starfield-Ln-89147/home/29557501',
'https://www.redfin.com/NV/Las-Vegas/8767-Crystal-Port-Ave-89147/home/29560610',
'https://www.redfin.com/NV/Las-Vegas/8501-W-University-Ave-89147/unit-2110/home/29560997',
'https://www.redfin.com/NV/Las-Vegas/7023-Forest-Vista-St-89147/home/29568085',
'https://www.redfin.com/NV/Las-Vegas/4383-Pineaire-St-89147/home/29568106',
'https://www.redfin.com/NV/Las-Vegas/7082-Acorn-Ct-89147/home/29568675',
'https://www.redfin.com/NV/Las-Vegas/7241-Girard-Dr-89147/home/29568703',
'https://www.redfin.com/NV/Las-Vegas/4749-Hackamore-Dr-89103/home/29570629',
'https://www.redfin.com/NV/Las-Vegas/6621-Shelter-Ln-89103/home/29570647',
'https://www.redfin.com/NV/Las-Vegas/6734-Wayburn-Way-89103/home/29570841',
'https://www.redfin.com/NV/Las-Vegas/6277-Fairbanks-Rd-89103/home/29571823',
'https://www.redfin.com/NV/Las-Vegas/4453-Ridgeville-St-89103/home/29572204',
'https://www.redfin.com/NV/Las-Vegas/6490-Gunderson-Blvd-89103/home/29572575',
'https://www.redfin.com/NV/Las-Vegas/6233-Bullion-Blvd-89103/home/29573247',
'https://www.redfin.com/NV/Las-Vegas/4659-Royal-Ridge-Blvd-89103/home/29575030',
'https://www.redfin.com/NV/Las-Vegas/4624-E-Royal-Club-Way-89103/home/29575387',
'https://www.redfin.com/NV/Las-Vegas/5237-Wave-Dancer-Ln-89118/home/29576578',
'https://www.redfin.com/NV/Las-Vegas/5226-Souvenir-Ln-89118/home/29576591',
'https://www.redfin.com/NV/Las-Vegas/5251-Lindell-Rd-89118/unit-205/home/29576887',
'https://www.redfin.com/NV/Las-Vegas/5263-Tropical-Peach-Dr-89118/home/29577204',
'https://www.redfin.com/NV/Las-Vegas/5330-Comstock-Lode-Ln-89118/home/29577440',
'https://www.redfin.com/NV/Las-Vegas/5756-Chisolm-Trl-89118/home/29577500',
'https://www.redfin.com/NV/Las-Vegas/5155-W-Tropicana-Ave-89103/unit-2175/home/29578267',
'https://www.redfin.com/NV/Las-Vegas/5229-Shasta-Lake-Way-89118/home/29579697',
'https://www.redfin.com/NV/Las-Vegas/5320-Sharon-Marie-Ct-89118/home/29579758',
'https://www.redfin.com/NV/Las-Vegas/6659-W-Tropicana-Ave-89103/unit-203/home/29579847',
'https://www.redfin.com/NV/Las-Vegas/6282-La-Palma-Pkwy-89118/home/29580121',
'https://www.redfin.com/NV/Las-Vegas/4914-Mission-Bay-Dr-89113/home/29580735',
'https://www.redfin.com/NV/Las-Vegas/6370-W-Dewey-Dr-89118/home/29580794',
'https://www.redfin.com/NV/Las-Vegas/7715-Top-Hat-Ave-89113/home/29581565',
'https://www.redfin.com/NV/Las-Vegas/5053-Mount-Pleasant-Ln-89113/home/29582195',
'https://www.redfin.com/NV/Las-Vegas/8271-Turtle-Creek-Cir-89113/home/29583121',
'https://www.redfin.com/NV/Las-Vegas/7825-Rancho-Mirage-Dr-89113/home/29583175',
'https://www.redfin.com/NV/Las-Vegas/5517-Morning-Swim-Ln-89113/home/29583970',
'https://www.redfin.com/NV/Las-Vegas/8555-W-Russell-Rd-89113/unit-1107/home/29584156',
'https://www.redfin.com/NV/Las-Vegas/7292-Purple-Shadow-Ave-89113/home/29585915',
'https://www.redfin.com/NV/Las-Vegas/10550-Cedar-Run-Ct-89135/home/29587749',
'https://www.redfin.com/NV/Las-Vegas/2890-Painted-Lilly-Dr-89135/home/29588864',
'https://www.redfin.com/NV/Las-Vegas/3249-River-Glorious-Ln-89135/home/29589401',
'https://www.redfin.com/NV/Las-Vegas/2700-Las-Vegas-Blvd-S-89109/unit-809/home/29784618',
'https://www.redfin.com/NV/Las-Vegas/2700-Las-Vegas-Blvd-S-89109/unit-505/home/29784676',
'https://www.redfin.com/NV/Las-Vegas/4381-W-Flamingo-Rd-89103/unit-5016/home/29789958',
'https://www.redfin.com/NV/Las-Vegas/4575-Dean-Martin-Dr-89103/unit-1407/home/29791283',
'https://www.redfin.com/NV/Las-Vegas/4575-Dean-Martin-Dr-89103/unit-3011/home/29792269',
'https://www.redfin.com/NV/Las-Vegas/4575-Dean-Martin-Dr-89103/unit-2107/home/29792288',
'https://www.redfin.com/NV/Las-Vegas/4471-Dean-Martin-Dr-89103/unit-801/home/29814037',
'https://www.redfin.com/NV/Las-Vegas/4471-Dean-Martin-Dr-89103/unit-3508/home/29815118',
'https://www.redfin.com/NV/Las-Vegas/8082-Planting-Fields-Pl-89117/home/29828876',
'https://www.redfin.com/NV/Las-Vegas/5055-W-Hacienda-Ave-89118/unit-2024/home/29829615',
'https://www.redfin.com/NV/Las-Vegas/5055-W-Hacienda-Ave-89118/unit-1068/home/29831069',
'https://www.redfin.com/NV/Las-Vegas/5055-W-Hacienda-Ave-89118/unit-1214/home/29831159',
'https://www.redfin.com/NV/Las-Vegas/5055-W-Hacienda-Ave-89118/unit-2041/home/29831278',
'https://www.redfin.com/NV/Las-Vegas/9895-Shadycrest-Ct-89148/home/29831330',
'https://www.redfin.com/NV/Las-Vegas/4424-San-Cascina-St-89135/home/29833228',
'https://www.redfin.com/NV/Las-Vegas/3050-Harbor-Heights-Dr-89117/home/29866688',
'https://www.redfin.com/NV/Las-Vegas/10094-San-Gervasio-Ave-89147/home/29867789',
'https://www.redfin.com/NV/Las-Vegas/4661-Altina-St-89147/home/29867814',
'https://www.redfin.com/NV/Las-Vegas/9907-Ridge-Manor-Ave-89148/home/29868756',
'https://www.redfin.com/NV/Las-Vegas/9778-W-Mesa-Vista-Ave-89148/home/29869423',
'https://www.redfin.com/NV/Las-Vegas/5902-Bellerive-St-89113/home/29870407',
'https://www.redfin.com/NV/Las-Vegas/8555-W-Russell-Rd-89113/unit-2057/home/29870560',
'https://www.redfin.com/NV/Las-Vegas/3562-Red-Fir-St-89135/home/29872337',
'https://www.redfin.com/NV/Las-Vegas/9674-Geiger-Peak-Ct-89148/home/29889880',
'https://www.redfin.com/NV/Las-Vegas/3143-Elk-Clover-St-89135/home/29896888',
'https://www.redfin.com/NV/Las-Vegas/10478-Melodia-Magico-Ave-89135/home/29897809',
'https://www.redfin.com/NV/Las-Vegas/10833-Silver-Lace-Ln-89135/home/29897933',
'https://www.redfin.com/NV/Las-Vegas/10267-Early-Morning-Ave-89135/home/29898052',
'https://www.redfin.com/NV/Las-Vegas/9578-Adobe-Arch-Ct-89148/home/29907188',
'https://www.redfin.com/NV/Las-Vegas/1659-Boundary-Peak-Way-89135/home/29908170',
'https://www.redfin.com/NV/Las-Vegas/1560-Boundary-Peak-Way-89135/home/29908374',
'https://www.redfin.com/NV/Las-Vegas/4530-Regalo-Bello-St-89135/home/29911988',
'https://www.redfin.com/NV/Las-Vegas/11072-Village-Ridge-Ln-89135/home/29916955',
'https://www.redfin.com/NV/Las-Vegas/10525-Bambola-Pl-89135/home/29917797',
'https://www.redfin.com/NV/Las-Vegas/35-Panorama-Crest-Ave-89135/home/29920056',
'https://www.redfin.com/NV/Las-Vegas/9495-Perennial-View-Ave-89148/home/29921080',
'https://www.redfin.com/NV/Las-Vegas/400-Rancho-La-Costa-St-89138/home/29931219',
'https://www.redfin.com/NV/Las-Vegas/3552-Ridge-Meadow-St-89135/home/29937336',
'https://www.redfin.com/NV/Las-Vegas/3750-Las-Vegas-Blvd-S-89158/unit-2406/home/30523235',
'https://www.redfin.com/NV/Las-Vegas/3726-Las-Vegas-Blvd-S-89158/unit-610/home/30534132',
'https://www.redfin.com/NV/Las-Vegas/3722-Las-Vegas-Blvd-S-89158/unit-1408/home/30534194',
'https://www.redfin.com/NV/Las-Vegas/10608-Tranquil-Glade-Ln-89135/home/35654738',
'https://www.redfin.com/NV/Las-Vegas/1870-Solvang-Mill-Dr-89135/home/40202891',
'https://www.redfin.com/NV/Las-Vegas/5623-Shadow-Bend-Dr-89135/home/49748897',
'https://www.redfin.com/NV/Las-Vegas/11362-Belmont-Lake-Dr-89135/unit-104/home/52622258',
'https://www.redfin.com/NV/Las-Vegas/8271-Buffalo-Ranch-Ave-89147/home/65557963',
'https://www.redfin.com/NV/Las-Vegas/8775-Cerrito-Canyon-Ct-89148/home/95953513',
'https://www.redfin.com/NV/Las-Vegas/81-Pristine-Glen-St-89135/home/101391854',
'https://www.redfin.com/NV/Las-Vegas/7714-Pyrenees-Park-Dr-89113/home/113035622',
'https://www.redfin.com/NV/Las-Vegas/8949-Brockhampton-Ct-89148/home/113713701',
'https://www.redfin.com/NV/Las-Vegas/5581-Cornadelo-Range-Rd-89113/home/144164027',
'https://www.redfin.com/NV/Las-Vegas/6474-Estrella-Hills-Ave-89118/home/144625082',
'https://www.redfin.com/NV/Las-Vegas/6360-Clovis-Point-St-89135/home/145294574',
'https://www.redfin.com/NV/Las-Vegas/11237-Rainbow-Peak-Ave-89135/unit-211/home/145873183',
'https://www.redfin.com/NV/Las-Vegas/11256-Rainbow-Peak-Ave-89135/unit-201/home/145873208',
'https://www.redfin.com/NV/Las-Vegas/5932-Wellington-Peak-Rd-89148/home/146417699',
'https://www.redfin.com/NV/Las-Vegas/4253-Sunrise-Flats-St-89135/home/146418250',
'https://www.redfin.com/NV/Las-Vegas/11436-Opal-Springs-Way-89135/home/160595682',
'https://www.redfin.com/NV/Las-Vegas/6216-Haiku-Stairs-St-89148/home/165915999',
'https://www.redfin.com/NV/Las-Vegas/4980-Vincitor-St-89135/home/28989513',
'https://www.redfin.com/NV/Las-Vegas/5619-Deer-Creek-Falls-Ct-89118/home/29031783',
'https://www.redfin.com/NV/Las-Vegas/6362-Whispering-Creek-St-89148/home/29032719',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-2711/home/29053260',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-2509/home/29053291',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-1802/home/29053372',
'https://www.redfin.com/NV/Las-Vegas/260-E-Flamingo-Rd-89169/unit-122/home/29064378',
'https://www.redfin.com/NV/Las-Vegas/5471-Indian-Cedar-Dr-89135/home/29065528',
'https://www.redfin.com/NV/Las-Vegas/10417-Orkiney-Dr-89144/home/29115187',
'https://www.redfin.com/NV/Las-Vegas/1021-Collingtree-St-89145/home/29213440',
'https://www.redfin.com/NV/Las-Vegas/7901-Aplin-Ave-89145/home/29218482',
'https://www.redfin.com/NV/Las-Vegas/105-Redstone-St-89145/home/29225641',
'https://www.redfin.com/NV/Las-Vegas/100-S-Crestline-Dr-89107/unit-12/home/29229155',
'https://www.redfin.com/NV/Las-Vegas/401-Mill-Hollow-Rd-89107/home/29235458',
'https://www.redfin.com/NV/Las-Vegas/5208-Alta-Dr-89107/home/29235759',
'https://www.redfin.com/NV/Las-Vegas/135-E-Harmon-Ave-89109/unit-804/home/29283549',
'https://www.redfin.com/NV/Las-Vegas/135-E-Harmon-Ave-89109/unit-221/home/29283658',
'https://www.redfin.com/NV/Las-Vegas/4224-Kay-Pl-89107/home/29343615',
'https://www.redfin.com/NV/Las-Vegas/4304-El-Conlon-Ave-89102/home/29463621',
'https://www.redfin.com/NV/Las-Vegas/211-E-Flamingo-Rd-89169/unit-715/home/29486967',
'https://www.redfin.com/NV/Las-Vegas/1430-Covelo-Ct-89146/home/29516479',
'https://www.redfin.com/NV/Las-Vegas/7481-Crystal-Forrest-Dr-89117/home/29520230',
'https://www.redfin.com/NV/Las-Vegas/1830-Casa-de-Elegante-Ct-89117/home/29520567',
'https://www.redfin.com/NV/Las-Vegas/8109-Via-del-Cerro-Ct-89117/home/29522285',
'https://www.redfin.com/NV/Las-Vegas/2541-Tumble-Brook-Dr-89134/home/28964528',
'https://www.redfin.com/NV/Las-Vegas/309-Gardenia-Ln-89107/home/28966001',
'https://www.redfin.com/NV/Las-Vegas/7200-Pirates-Cove-Rd-89145/unit-1103/home/28969570',
'https://www.redfin.com/NV/Las-Vegas/144-Macoby-Run-St-89148/home/28972126',
'https://www.redfin.com/NV/Las-Vegas/5611-Aurora-Crest-Ave-89139/home/28974208',
'https://www.redfin.com/NV/Las-Vegas/9667-Brooks-Lake-Ave-89148/home/28976812',
'https://www.redfin.com/NV/Las-Vegas/3898-Chase-Glen-Cir-89121/home/28976892',
'https://www.redfin.com/NV/Las-Vegas/7595-Ironwood-Knoll-Ave-89113/home/28978709',
'https://www.redfin.com/NV/Las-Vegas/7967-Delft-Blue-St-89139/home/28991574',
'https://www.redfin.com/NV/Las-Vegas/4839-Jubilee-Diamond-Ct-89139/home/29005868',
'https://www.redfin.com/NV/Las-Vegas/5848-Old-English-Ct-89139/home/29013361',
'https://www.redfin.com/NV/Las-Vegas/6255-Rock-Dove-Ave-89122/home/29029357',
'https://www.redfin.com/NV/Las-Vegas/3145-E-Flamingo-Rd-89121/unit-2105/home/29029740',
'https://www.redfin.com/NV/Las-Vegas/7702-Hampton-Willows-Ln-89113/home/29034777',
'https://www.redfin.com/NV/Las-Vegas/5741-Old-Colony-Dr-89139/home/29036208',
'https://www.redfin.com/NV/Las-Vegas/4989-Tioga-Pass-Ave-89139/home/29036350',
'https://www.redfin.com/NV/Las-Vegas/9216-Cazador-St-89123/home/29039308',
'https://www.redfin.com/NV/Las-Vegas/2451-N-Rainbow-Blvd-89108/unit-2110/home/29062106',
'https://www.redfin.com/NV/Las-Vegas/2451-N-Rainbow-Blvd-89108/unit-1073/home/29062176',
'https://www.redfin.com/NV/Las-Vegas/7411-Bedford-Pine-Ct-89113/home/29066287',
'https://www.redfin.com/NV/Las-Vegas/8994-Harmony-Grove-Ave-89148/home/29067605',
'https://www.redfin.com/NV/Las-Vegas/10605-Back-Plains-Dr-89134/home/29105003',
'https://www.redfin.com/NV/Las-Vegas/10801-Heritage-Hills-Dr-89134/home/29107513',
'https://www.redfin.com/NV/Las-Vegas/10117-Cardinal-View-Pl-89134/home/29109306',
'https://www.redfin.com/NV/Las-Vegas/10303-Sawmill-Ave-89134/home/29110005',
'https://www.redfin.com/NV/Las-Vegas/5613-Deodar-Dr-89108/home/29136185',
'https://www.redfin.com/NV/Las-Vegas/6625-Old-Oxford-Ave-89108/home/29137874',
'https://www.redfin.com/NV/Las-Vegas/6228-Heather-Mist-Ln-89108/home/29140191',
'https://www.redfin.com/NV/Las-Vegas/3049-Brigantine-Way-89128/home/29141154',
'https://www.redfin.com/NV/Las-Vegas/8125-Sapphire-Bay-Cir-89128/home/29142773',
'https://www.redfin.com/NV/Las-Vegas/2417-Sun-Shores-Dr-89128/home/29143904',
'https://www.redfin.com/NV/Las-Vegas/2648-Golden-Sands-Dr-89128/home/29146127',
'https://www.redfin.com/NV/Las-Vegas/2653-Seahorse-Ct-89128/home/29146145',
'https://www.redfin.com/NV/Las-Vegas/2656-Echo-Mesa-Dr-89134/home/29148092',
'https://www.redfin.com/NV/Las-Vegas/10033-Woodhouse-Dr-89134/home/29152031',
'https://www.redfin.com/NV/Las-Vegas/9748-Panorama-Cliff-Dr-89134/home/29156068',
'https://www.redfin.com/NV/Las-Vegas/1900-Mountain-Hills-Ct-89128/unit-202/home/29165238',
'https://www.redfin.com/NV/Las-Vegas/7925-Sierra-Largo-Dr-89128/home/29165298',
'https://www.redfin.com/NV/Las-Vegas/7820-Desert-Bell-Ave-89128/home/29166303',
'https://www.redfin.com/NV/Las-Vegas/1636-Mexican-Poppy-St-89128/home/29166928',
'https://www.redfin.com/NV/Las-Vegas/7212-Old-Mission-Dr-89128/home/29168865',
'https://www.redfin.com/NV/Las-Vegas/1800-Leaning-Pine-Way-89128/home/29170090',
'https://www.redfin.com/NV/Las-Vegas/2136-Willowbury-Dr-89108/unit-A/home/29170751',
'https://www.redfin.com/NV/Las-Vegas/1608-Burningwood-Ln-89108/unit-101/home/29172228',
'https://www.redfin.com/NV/Las-Vegas/6220-Yerba-Ln-89108/home/29173481',
'https://www.redfin.com/NV/Las-Vegas/5624-Eugene-Ave-89108/home/29175561',
'https://www.redfin.com/NV/Las-Vegas/5921-Pebble-Beach-Blvd-89108/home/29185102',
'https://www.redfin.com/NV/Las-Vegas/5213-Cannon-Blvd-89108/home/29189285',
'https://www.redfin.com/NV/Las-Vegas/6516-Moon-Roses-Ct-89108/home/29192022',
'https://www.redfin.com/NV/Las-Vegas/1021-Dreamland-Cir-89108/home/29195027',
'https://www.redfin.com/NV/Las-Vegas/1112-Spunsilk-Cir-89108/home/29195282',
'https://www.redfin.com/NV/Las-Vegas/1405-Grangeville-Dr-89108/home/29195326',
'https://www.redfin.com/NV/Las-Vegas/6304-Clarice-Ave-89107/home/29197015',
'https://www.redfin.com/NV/Las-Vegas/1150-N-Buffalo-Dr-89128/unit-1056/home/29199054',
'https://www.redfin.com/NV/Las-Vegas/7100-Pirates-Cove-Rd-89145/unit-1062/home/29200431',
'https://www.redfin.com/NV/Las-Vegas/7001-Alamosa-Way-89128/home/29201284',
'https://www.redfin.com/NV/Las-Vegas/8033-Celestial-Ave-89128/unit-201/home/29202804',
'https://www.redfin.com/NV/Las-Vegas/8021-Cimarron-Ridge-Dr-89128/unit-201/home/29203820',
'https://www.redfin.com/NV/Las-Vegas/7933-Decker-Canyon-Dr-89128/unit-202/home/29205629',
'https://www.redfin.com/NV/Las-Vegas/1312-Driscoll-Dr-89128/unit-202/home/29206525',
'https://www.redfin.com/NV/Las-Vegas/10116-Jacob-Pl-89144/unit-203/home/29209757',
'https://www.redfin.com/NV/Las-Vegas/9712-Amber-Peak-Ct-89144/home/29210466',
'https://www.redfin.com/NV/Las-Vegas/112-Tesoro-Dr-89144/home/29210597',
'https://www.redfin.com/NV/Las-Vegas/9208-Sunnyfield-Dr-89134/home/29211532',
'https://www.redfin.com/NV/Las-Vegas/5395-Raccoon-Valley-Ln-89122/home/29242720',
'https://www.redfin.com/NV/Las-Vegas/7701-W-Robindale-Rd-89113/unit-164/home/29287192',
'https://www.redfin.com/NV/Las-Vegas/8985-S-Durango-Dr-89113/unit-2059/home/29290707',
'https://www.redfin.com/NV/Las-Vegas/9517-Dawning-Heat-St-89178/home/29290807',
'https://www.redfin.com/NV/Las-Vegas/8256-River-Beach-Ave-89178/home/29292576',
'https://www.redfin.com/NV/Las-Vegas/4616-Eugene-Ave-89108/home/29305235',
'https://www.redfin.com/NV/Las-Vegas/2112-Havelina-St-89108/home/29305300',
'https://www.redfin.com/NV/Las-Vegas/4516-W-Lake-Mead-Blvd-89108/unit-102/home/29306176',
'https://www.redfin.com/NV/Las-Vegas/2213-Saxton-Hill-Ave-89106/home/29309694',
'https://www.redfin.com/NV/Las-Vegas/2520-Perryville-Ave-89106/unit-103/home/29310639',
'https://www.redfin.com/NV/Las-Vegas/5424-Bradford-Pear-Dr-89122/home/29315558',
'https://www.redfin.com/NV/Las-Vegas/1037-Lawry-Ave-89106/home/29323847',
'https://www.redfin.com/NV/Las-Vegas/4405-Saint-Andrews-Cir-89107/home/29340815',
'https://www.redfin.com/NV/Las-Vegas/2725-S-Nellis-Blvd-89121/unit-1197/home/29414678',
'https://www.redfin.com/NV/Las-Vegas/2725-S-Nellis-Blvd-89121/unit-2185/home/29414778',
'https://www.redfin.com/NV/Las-Vegas/4864-Rollingwood-Cir-89121/home/29426804',
'https://www.redfin.com/NV/Las-Vegas/5361-Simpson-Dr-89122/home/29431493',
'https://www.redfin.com/NV/Las-Vegas/5710-E-Tropicana-Ave-89122/unit-1177/home/29431744',
'https://www.redfin.com/NV/Las-Vegas/5710-E-Tropicana-Ave-89122/unit-1090/home/29431976',
'https://www.redfin.com/NV/Las-Vegas/5187-Midnight-Oil-Dr-89122/home/29434404',
'https://www.redfin.com/NV/Las-Vegas/5375-Shake-Ct-89122/unit-203/home/29435391',
'https://www.redfin.com/NV/Las-Vegas/5040-Haystack-Dr-89122/home/29435787',
'https://www.redfin.com/NV/Las-Vegas/5624-Wellesley-Dr-89122/home/29437629',
'https://www.redfin.com/NV/Las-Vegas/5439-Northridge-Ln-89122/home/29437922',
'https://www.redfin.com/NV/Las-Vegas/4855-Terra-Linda-Ave-89120/home/29440434',
'https://www.redfin.com/NV/Las-Vegas/5327-Lamoille-Cir-89120/home/29442155',
'https://www.redfin.com/NV/Las-Vegas/9300-Hosner-St-89178/home/29490136',
'https://www.redfin.com/NV/Las-Vegas/1300-Lorilyn-Ave-89119/unit-1/home/29505862',
'https://www.redfin.com/NV/Las-Vegas/4770-Topaz-St-89121/unit-57/home/29508187',
'https://www.redfin.com/NV/Las-Vegas/5447-Birchbrook-Ct-89120/home/29511589',
'https://www.redfin.com/NV/Las-Vegas/5216-Mandalay-Springs-Dr-89120/unit-101/home/29511813',
'https://www.redfin.com/NV/Las-Vegas/873-De-Met-Dr-89119/home/29515357',
'https://www.redfin.com/NV/Las-Vegas/1107-Westminster-Ave-89119/home/29516255',
'https://www.redfin.com/NV/Las-Vegas/1067-Westminster-Ave-89119/home/29516278',
'https://www.redfin.com/NV/Las-Vegas/3286-E-Oquendo-Rd-89120/home/29516745',
'https://www.redfin.com/NV/Las-Vegas/1770-Autumn-Rust-Dr-89119/home/29592403',
'https://www.redfin.com/NV/Las-Vegas/6992-Antell-Cir-89119/home/29592494',
'https://www.redfin.com/NV/Las-Vegas/1430-Recital-Way-89119/home/29592607',
'https://www.redfin.com/NV/Las-Vegas/7049-Morales-Cir-89119/home/29593635',
'https://www.redfin.com/NV/Las-Vegas/1730-Forest-Walk-Dr-89119/home/29593744',
'https://www.redfin.com/NV/Las-Vegas/7078-Knoll-View-Dr-89119/home/29593753',
'https://www.redfin.com/NV/Las-Vegas/7132-Paradise-Bay-Dr-89119/home/29594041',
'https://www.redfin.com/NV/Las-Vegas/7026-Troubador-Dr-89119/home/29594241',
'https://www.redfin.com/NV/Las-Vegas/6650-W-Warm-Springs-Rd-89118/unit-1176/home/29603979',
'https://www.redfin.com/NV/Las-Vegas/9308-Night-Mesa-St-89178/home/29606326',
'https://www.redfin.com/NV/Las-Vegas/490-E-Mardon-Ave-89123/home/29606896',
'https://www.redfin.com/NV/Las-Vegas/9777-Plane-Tree-Ct-89178/home/29607481',
'https://www.redfin.com/NV/Las-Vegas/950-Elysian-Dr-89123/home/29611623',
'https://www.redfin.com/NV/Las-Vegas/7450-S-Eastern-Ave-89123/unit-1004/home/29614448',
'https://www.redfin.com/NV/Las-Vegas/7450-S-Eastern-Ave-89123/unit-1051/home/29614591',
'https://www.redfin.com/NV/Las-Vegas/653-Almondale-Ave-89123/home/29622953',
'https://www.redfin.com/NV/Las-Vegas/578-Lennox-Dr-89123/home/29623763',
'https://www.redfin.com/NV/Las-Vegas/392-E-Shelbourne-Ave-89123/home/29626434',
'https://www.redfin.com/NV/Las-Vegas/130-E-Wigwam-Ave-89123/home/29626636',
'https://www.redfin.com/NV/Las-Vegas/9147-Briarthorne-St-89123/home/29629425',
'https://www.redfin.com/NV/Las-Vegas/9394-Graceful-Gold-St-89123/home/29632765',
'https://www.redfin.com/NV/Las-Vegas/1466-October-Oak-Ave-89123/home/29633592',
'https://www.redfin.com/NV/Las-Vegas/9836-Altadena-St-89183/home/29638570',
'https://www.redfin.com/NV/Las-Vegas/684-Gritty-Garnet-Ave-89183/home/29638814',
'https://www.redfin.com/NV/Las-Vegas/9320-Column-Cactus-St-89139/home/29770079',
'https://www.redfin.com/NV/Las-Vegas/6694-Chimes-Tower-Ave-89139/home/29834513',
'https://www.redfin.com/NV/Las-Vegas/3209-Turtle-Head-Peak-Dr-89135/home/29870712',
'https://www.redfin.com/NV/Las-Vegas/492-Kirkstone-Way-89123/home/29875134',
'https://www.redfin.com/NV/Las-Vegas/172-Staten-Island-Ave-89123/home/29875245',
'https://www.redfin.com/NV/Las-Vegas/9963-Fine-Fern-St-89183/home/29876920',
'https://www.redfin.com/NV/Las-Vegas/7243-Twin-Maples-Ct-89148/home/29898432',
'https://www.redfin.com/NV/Las-Vegas/173-Duck-Hollow-Ave-89148/home/29908380',
'https://www.redfin.com/NV/Las-Vegas/7590-Bear-River-Ct-89139/home/29939357',
'https://www.redfin.com/NV/Las-Vegas/9451-Golden-Bell-Dr-89139/home/30496436',
'https://www.redfin.com/NV/Las-Vegas/8445-Insignia-Ave-89178/unit-105/home/33373982',
'https://www.redfin.com/NV/Las-Vegas/62-Tribute-Peak-Way-89148/home/40234276',
'https://www.redfin.com/NV/Las-Vegas/9022-Winthrop-Springs-Rd-89139/home/40548087',
'https://www.redfin.com/NV/Las-Vegas/6423-W-Camero-Ave-89139/home/52625580',
'https://www.redfin.com/NV/Las-Vegas/1515-E-Reno-Ave-89119/unit-203/home/77433410',
'https://www.redfin.com/NV/Las-Vegas/6092-Crown-Palms-Ave-89139/home/92907917',
'https://www.redfin.com/NV/Las-Vegas/6760-Goose-Watch-Ct-89139/home/101992606',
'https://www.redfin.com/NV/Las-Vegas/6367-Beavertail-Pond-Ave-89122/home/109632554',
'https://www.redfin.com/NV/Las-Vegas/9049-Janice-Glen-Ave-89148/home/114093564',
'https://www.redfin.com/NV/Las-Vegas/6325-Point-Isabel-Way-89122/home/143188514',
'https://www.redfin.com/NV/Las-Vegas/4936-Quest-Tribe-St-89122/home/146084229',
'https://www.redfin.com/NV/Las-Vegas/9240-Black-Diamond-St-89178/home/161752134',
'https://www.redfin.com/NV/Las-Vegas/11280-Granite-Ridge-Dr-89135/unit-1006/home/165926155',
'https://www.redfin.com/NV/Las-Vegas/9746-W-Richmar-Ave-89178/home/167760986',
'https://www.redfin.com/NV/Las-Vegas/9196-Lynea-Ct-89178/home/167769246',
'https://www.redfin.com/NV/Las-Vegas/5400-Swenson-St-89119/unit-32/home/167955788',
'https://www.redfin.com/NV/Las-Vegas/6295-Whippet-Ave-89122/home/169230639',
'https://www.redfin.com/NV/Las-Vegas/5158-Peak-Climb-Dr-89135/home/169231663',
'https://www.redfin.com/NV/Las-Vegas/7809-Waterfalls-Ave-89128/home/171652555',
'https://www.redfin.com/NV/Las-Vegas/7110-Cass-Ave-89113/home/175525674',
'https://www.redfin.com/NV/Las-Vegas/7898-W-Agate-Ave-89113/unit-65/home/185869640',
'https://www.redfin.com/NV/Enterprise/4515-Ashley-Hills-Ave-89141/unit-27/home/186207631',
'https://www.redfin.com/NV/Las-Vegas/7925-Whisper-Words-Ave-89113/unit-37/home/186301980',
'https://www.redfin.com/NV/Enterprise/4499-Ashley-Hills-Ave-89141/unit-23/home/186302872',
'https://www.redfin.com/NV/Enterprise/9930-Florence-Hills-St-89141/unit-116/home/186636704',
'https://www.redfin.com/NV/Las-Vegas/9128-Rocky-Raccoon-ST-89113/unit-19/home/186902560',
'https://www.redfin.com/NV/Las-Vegas/7869-Sailor-Sam-Ave-89113/unit-24/home/186958351',
'https://www.redfin.com/NV/Las-Vegas/9762-Sage-Grove-Ct-89148/home/29032846',
'https://www.redfin.com/NV/Las-Vegas/5441-Scotch-Cap-Dr-89122/home/145870780',
'https://www.redfin.com/NV/Las-Vegas/8457-Insignia-Ave-89178/unit-102/home/147856117',
'https://www.redfin.com/NV/Las-Vegas/5282-Belmont-Mill-Ct-89122/home/161754850',
'https://www.redfin.com/NV/Las-Vegas/11280-Granite-Ridge-Dr-89135/unit-1023/home/166555086',
'https://www.redfin.com/NV/Las-Vegas/11238-Essence-Point-Ave-89135/unit-203/home/166601089',
'https://www.redfin.com/NV/Las-Vegas/5614-Calm-Season-St-89118/home/167745194',
'https://www.redfin.com/NV/Las-Vegas/7493-Triple-Knot-Ct-89113/home/170466528',
'https://www.redfin.com/NV/Las-Vegas/4228-Bayley-Skye-Ave-89141/home/173636482',
'https://www.redfin.com/NV/Las-Vegas/6244-Yerba-Ln-89108/home/183394749',
'https://www.redfin.com/NV/Enterprise/9935-Florence-Hills-St-89141/unit-107/home/186275806',
'https://www.redfin.com/NV/Las-Vegas/1375-E-Hacienda-Ave-89119/unit-105/home/29513412',
'https://www.redfin.com/NV/Las-Vegas/6165-Meadow-View-Ln-89103/home/29545653',
'https://www.redfin.com/NV/Las-Vegas/4909-Quiet-Morning-St-89122/home/166181309',
'https://www.redfin.com/NV/Las-Vegas/6820-Scarlet-Flax-St-89148/home/28972475',
'https://www.redfin.com/NV/Las-Vegas/5411-Pinosa-Ct-89141/home/28975799',
'https://www.redfin.com/NV/Las-Vegas/11424-Glowing-Sunset-Ln-89135/home/28989084',
'https://www.redfin.com/NV/Las-Vegas/11072-Pentland-Downs-St-89141/home/28992867',
'https://www.redfin.com/NV/Las-Vegas/11690-Vacanze-Ct-89183/home/28994826',
'https://www.redfin.com/NV/Las-Vegas/128-Struzzo-Ave-89183/home/28994856',
'https://www.redfin.com/NV/Las-Vegas/8465-Adams-Grove-St-89139/home/29007763',
'https://www.redfin.com/NV/Las-Vegas/6239-Bedding-Plane-Ct-89139/home/29010782',
'https://www.redfin.com/NV/Las-Vegas/6255-Peneplain-Ave-89139/home/29035228',
'https://www.redfin.com/NV/Las-Vegas/137-Water-Hazard-Ln-89148/home/29037651',
'https://www.redfin.com/NV/Las-Vegas/10288-Nolinas-St-89141/home/29038486',
'https://www.redfin.com/NV/Las-Vegas/10819-Vestone-St-89141/home/29038527',
'https://www.redfin.com/NV/Las-Vegas/10623-Lessona-St-89141/home/29038650',
'https://www.redfin.com/NV/Las-Vegas/10864-Fishers-Island-St-89141/home/29040149',
'https://www.redfin.com/NV/Las-Vegas/9158-Placer-Bullion-Ave-89178/home/29055222',
'https://www.redfin.com/NV/Las-Vegas/10082-Amber-Field-St-89178/home/29055927',
'https://www.redfin.com/NV/Las-Vegas/7255-W-Sunset-Rd-89113/unit-2124/home/29065894',
'https://www.redfin.com/NV/Las-Vegas/8000-Badura-Ave-89113/unit-1173/home/29066604',
'https://www.redfin.com/NV/Las-Vegas/8767-Brilliant-Star-Dr-89178/home/29068643',
'https://www.redfin.com/NV/Las-Vegas/10315-Eve-Springs-St-89178/home/29069378',
'https://www.redfin.com/NV/Las-Vegas/2300-E-Silverado-Ranch-Blvd-89183/unit-2061/home/29070141',
'https://www.redfin.com/NV/Las-Vegas/555-E-Silverado-Ranch-Blvd-89183/unit-2089/home/29070464',
'https://www.redfin.com/NV/Las-Vegas/7701-W-Robindale-Rd-89113/unit-226/home/29287225',
'https://www.redfin.com/NV/Las-Vegas/5040-Upper-Falls-Ct-89141/home/29291877',
'https://www.redfin.com/NV/Las-Vegas/9037-Castledowns-St-89148/home/29321168',
'https://www.redfin.com/NV/Las-Vegas/3851-Topaz-St-89121/home/29474075',
'https://www.redfin.com/NV/Las-Vegas/3110-Espanol-Dr-89121/home/29475684',
'https://www.redfin.com/NV/Las-Vegas/1455-E-Katie-Ave-89119/unit-D26/home/29483425',
'https://www.redfin.com/NV/Las-Vegas/8976-Lily-Touchstone-Ct-89148/home/29489724',
'https://www.redfin.com/NV/Las-Vegas/5553-Markley-Ave-89141/home/29490147',
'https://www.redfin.com/NV/Las-Vegas/3076-Bella-Verona-Ave-89141/home/29495849',
'https://www.redfin.com/NV/Las-Vegas/3930-University-Center-Dr-89119/unit-910/home/29501079',
'https://www.redfin.com/NV/Las-Vegas/1569-Hialeah-Dr-89119/home/29504833',
'https://www.redfin.com/NV/Las-Vegas/1386-Lorilyn-Ave-89119/unit-4/home/29505898',
'https://www.redfin.com/NV/Las-Vegas/1700-Gabriel-Dr-89119/home/29505915',
'https://www.redfin.com/NV/Las-Vegas/2007-Rockledge-Dr-89119/home/29507296',
'https://www.redfin.com/NV/Las-Vegas/4584-Casa-Mia-Cir-89121/home/29507756',
'https://www.redfin.com/NV/Las-Vegas/2565-Malabar-Ave-89121/home/29507845',
'https://www.redfin.com/NV/Las-Vegas/4770-Topaz-St-89121/unit-22/home/29508291',
'https://www.redfin.com/NV/Las-Vegas/2431-Paradise-Village-Way-89120/home/29509942',
'https://www.redfin.com/NV/Las-Vegas/5259-Caspian-Springs-Dr-89120/unit-204/home/29511038',
'https://www.redfin.com/NV/Las-Vegas/3215-Casey-Dr-89120/unit-204/home/29511080',
'https://www.redfin.com/NV/Las-Vegas/8475-Gagnier-Blvd-89113/home/29591832',
'https://www.redfin.com/NV/Las-Vegas/1677-Garden-Path-Ct-89119/home/29592389',
'https://www.redfin.com/NV/Las-Vegas/6010-Sierra-Medina-Ave-89139/home/29606150',
'https://www.redfin.com/NV/Las-Vegas/9992-Big-Sur-Mountain-St-89178/home/29606926',
'https://www.redfin.com/NV/Las-Vegas/6406-Fulton-Meadows-Ave-89141/home/29608844',
'https://www.redfin.com/NV/Las-Vegas/1586-Laurel-Oak-Dr-89123/home/29613664',
'https://www.redfin.com/NV/Las-Vegas/1555-Yellowwood-Dr-89123/home/29613712',
'https://www.redfin.com/NV/Las-Vegas/722-Irwindale-Ave-89123/home/29622284',
'https://www.redfin.com/NV/Las-Vegas/9462-Boulder-Creek-St-89123/home/29627883',
'https://www.redfin.com/NV/Las-Vegas/868-Rainbow-Rock-St-89123/home/29631268',
'https://www.redfin.com/NV/Las-Vegas/1865-Silver-Cactus-Ct-89183/home/29637565',
'https://www.redfin.com/NV/Las-Vegas/8255-Las-Vegas-Blvd-S-89123/unit-121/home/29800376',
'https://www.redfin.com/NV/Las-Vegas/8255-Las-Vegas-Blvd-S-89123/unit-505/home/29800446',
'https://www.redfin.com/NV/Las-Vegas/10578-Gambassi-Ct-89141/home/29823866',
'https://www.redfin.com/NV/Las-Vegas/46-Perennial-Border-Ct-89148/home/29833521',
'https://www.redfin.com/NV/Las-Vegas/7810-Nautilus-Shell-St-89139/home/29834340',
'https://www.redfin.com/NV/Las-Vegas/976-Pagoda-Tree-Ct-89183/home/29837112',
'https://www.redfin.com/NV/Las-Vegas/1050-E-Cactus-Ave-89183/unit-1034/home/29877282',
'https://www.redfin.com/NV/Las-Vegas/1050-E-Cactus-Ave-89183/unit-1045/home/29878157',
'https://www.redfin.com/NV/Las-Vegas/5644-Bracana-Ct-89141/home/29900427',
'https://www.redfin.com/NV/Las-Vegas/7977-Marble-City-Ct-89139/home/29901549',
'https://www.redfin.com/NV/Las-Vegas/3722-Caesars-Cir-89120/home/29903189',
'https://www.redfin.com/NV/Las-Vegas/10613-Eagle-Nest-St-89141/home/29909775',
'https://www.redfin.com/NV/Las-Vegas/10626-Eagle-Nest-St-89141/home/29910746',
'https://www.redfin.com/NV/Las-Vegas/10257-Bethel-Mill-St-89183/home/29917816',
'https://www.redfin.com/NV/Las-Vegas/7482-Aurora-Glow-St-89139/home/29939327',
'https://www.redfin.com/NV/Las-Vegas/5578-Casa-Palazzo-Ct-89141/home/29940313',
'https://www.redfin.com/NV/Las-Vegas/3049-Scalise-Ct-89141/home/29941825',
'https://www.redfin.com/NV/Las-Vegas/9325-Wild-Lariat-Ave-89178/home/30301948',
'https://www.redfin.com/NV/Las-Vegas/10016-Arwen-St-89178/home/30476230',
'https://www.redfin.com/NV/Las-Vegas/31-Bay-Course-Ct-89148/home/39609547',
'https://www.redfin.com/NV/Las-Vegas/11216-Jewel-Desert-Ct-89179/home/40219853',
'https://www.redfin.com/NV/Las-Vegas/10355-Copalito-Dr-89178/home/44585152',
'https://www.redfin.com/NV/Las-Vegas/342-Bridgeton-Cross-Ct-89148/home/49748640',
'https://www.redfin.com/NV/Las-Vegas/10236-Mopan-Rd-89178/home/52594043',
'https://www.redfin.com/NV/Las-Vegas/75-Buckthorn-Ridge-Ct-89183/home/52629721',
'https://www.redfin.com/NV/Las-Vegas/8917-Fox-Season-Ave-89178/home/92905366',
'https://www.redfin.com/NV/Las-Vegas/7583-Mallard-Bay-Ave-89179/home/109630406',
'https://www.redfin.com/NV/Las-Vegas/9055-Linden-Oak-St-89178/home/109633075',
'https://www.redfin.com/NV/Las-Vegas/3031-Versace-Ave-89141/home/109635400',
'https://www.redfin.com/NV/Las-Vegas/61-Olympia-Canyon-Way-89141/home/112855191',
'https://www.redfin.com/NV/Las-Vegas/8053-Green-River-Knob-St-89113/home/113712910',
'https://www.redfin.com/NV/Las-Vegas/7670-Monomoy-Bay-Ave-89179/home/143188810',
'https://www.redfin.com/NV/Las-Vegas/12239-Pro-Tour-Ct-89141/home/144686982',
'https://www.redfin.com/NV/Las-Vegas/102-Opulent-Rose-Ave-89183/home/147875410',
'https://www.redfin.com/NV/Las-Vegas/10860-Woods-Hole-Bay-St-89179/home/161713664',
'https://www.redfin.com/NV/Las-Vegas/12-Olympia-Ridge-Dr-89141/home/166057757',
'https://www.redfin.com/NV/Las-Vegas/12817-Slipknot-St-89141/home/166058055',
'https://www.redfin.com/NV/Las-Vegas/12888-Slipknot-St-89141/home/166438688',
'https://www.redfin.com/NV/Las-Vegas/10735-Galveston-Bay-St-89179/home/166605131',
'https://www.redfin.com/NV/Las-Vegas/10142-Eden-Mountain-St-89141/home/177143398',
'https://www.redfin.com/NV/Las-Vegas/11765-Discovery-Canyon-Dr-89135/home/181143275',
'https://www.redfin.com/NV/Las-Vegas/10112-Queens-Pearl-St-89141/home/182477817',
'https://www.redfin.com/NV/Las-Vegas/10118-Queens-Pearl-St-89141/home/182477818',
'https://www.redfin.com/NV/Las-Vegas/10279-Fragrant-Apple-Ct-89141/home/184024803',
'https://www.redfin.com/NV/Las-Vegas/10149-Cluny-Ave-89178/home/171321415',
'https://www.redfin.com/NV/Las-Vegas/9508-Medway-Towns-Ave-89178/home/172505844',
'https://www.redfin.com/NV/Las-Vegas/10389-Arroyo-Willow-St-89141/home/173454664',
'https://www.redfin.com/NV/Las-Vegas/9186-Theodore-Ct-89178/home/175526787',
'https://www.redfin.com/NV/Las-Vegas/10897-Lexus-Ct-89141/home/175527139',
'https://www.redfin.com/NV/Las-Vegas/234-Seven-Valleys-Ct-89183/home/175528772',
'https://www.redfin.com/NV/Las-Vegas/10250-Hairstreak-St-89141/home/178675152',
'https://www.redfin.com/NV/Las-Vegas/3965-Speckled-Wood-Ave-89141/home/178675168',
'https://www.redfin.com/NV/Las-Vegas/3465-Cosima-Ln-89120/home/179977221',
'https://www.redfin.com/NV/Las-Vegas/4472-Hidden-Stone-Ave-89141/home/182477813',
'https://www.redfin.com/NV/Las-Vegas/10124-Queens-Pearl-St-89141/home/182477819',
'https://www.redfin.com/NV/Las-Vegas/5262-Peak-Rock-Ct-89141/home/182484526',
'https://www.redfin.com/NV/Las-Vegas/9554-Dollardton-St-89139/home/183047229',
'https://www.redfin.com/NV/Las-Vegas/6219-Runestone-ST-89113/unit-19/home/185714454',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-3419-3421/home/186231679',
'https://www.redfin.com/NV/Las-Vegas/6211-Runestone-St-89113/unit-21/home/186339656',
'https://www.redfin.com/NV/Las-Vegas/145-E-Harmon-Ave-89109/unit-1801-1803/home/186342905',
'https://www.redfin.com/NV/Las-Vegas/5681-Steampunk-ST-89118/unit-13/home/186520991',
'https://www.redfin.com/NV/Las-Vegas/4120-Yucca-Bloom-St-89135/unit-258/home/186853214',
'https://www.redfin.com/NV/Las-Vegas/242-Fairway-Woods-Dr-89148/home/29014256',
'https://www.redfin.com/NV/Las-Vegas/6072-Lamotte-Ave-89141/home/29038542',
'https://www.redfin.com/NV/Las-Vegas/3750-Pilgrim-St-89121/home/29423252',
'https://www.redfin.com/NV/Las-Vegas/3089-Casey-Dr-89120/unit-202/home/29511948',
'https://www.redfin.com/NV/Las-Vegas/8925-Waymire-Creek-Ct-89147/home/29554095',
'https://www.redfin.com/NV/Las-Vegas/9990-Clear-Crystal-St-89183/home/29638723',
'https://www.redfin.com/NV/Las-Vegas/10675-Moss-Lake-St-89179/home/40234006',
'https://www.redfin.com/NV/Henderson/3136-Floral-Vista-Ave-89014/home/28963885',
'https://www.redfin.com/NV/Las-Vegas/6250-W-Arby-Ave-89118/unit-140/home/28964396',
'https://www.redfin.com/NV/Henderson/783-Rise-Canyon-Dr-89052/home/28972337',
'https://www.redfin.com/NV/Las-Vegas/4704-Sequoia-Park-Ave-89139/home/28978938',
'https://www.redfin.com/NV/Las-Vegas/6648-Oxendale-Ave-89139/home/28990998',
'https://www.redfin.com/NV/Las-Vegas/6692-Dunraven-Ave-89139/home/28991132',
'https://www.redfin.com/NV/Las-Vegas/6692-Hathersage-Ave-89139/home/28991302',
'https://www.redfin.com/NV/Las-Vegas/6629-Hurkling-Stone-Ave-89139/home/28991356',
'https://www.redfin.com/NV/Las-Vegas/6254-Arbor-Valley-Ave-89139/home/28992303',
'https://www.redfin.com/NV/Las-Vegas/11778-Weybrook-Park-Dr-89141/home/28995386',
'https://www.redfin.com/NV/Las-Vegas/118-Tansy-Ct-89183/home/28996606',
'https://www.redfin.com/NV/Las-Vegas/7397-Laramie-Ave-89113/home/28997827',
'https://www.redfin.com/NV/Las-Vegas/5031-Pine-Mountain-Ave-89139/home/29004067',
'https://www.redfin.com/NV/Henderson/1054-Astounding-Hills-Dr-89052/home/29005317',
'https://www.redfin.com/NV/Henderson/10843-Colour-Magic-St-89052/home/29005866',
'https://www.redfin.com/NV/Henderson/1052-Augusta-Wood-Ct-89052/home/29007575',
'https://www.redfin.com/NV/Henderson/311-Bass-Dr-89014/unit-D/home/29009036',
'https://www.redfin.com/NV/Henderson/2161-Big-Bar-Dr-89052/home/29012589',
'https://www.redfin.com/NV/Henderson/11128-Abbeyfield-Rose-Dr-89052/home/29014548',
'https://www.redfin.com/NV/Las-Vegas/6209-Alpine-Tree-Ave-89139/home/29015685',
'https://www.redfin.com/NV/Las-Vegas/7616-Poppy-Springs-Ave-89113/home/29034618',
'https://www.redfin.com/NV/Las-Vegas/7264-Larix-Rd-89113/home/29035050',
'https://www.redfin.com/NV/Las-Vegas/7158-Barton-Creek-Ct-89113/home/29035582',
'https://www.redfin.com/NV/Las-Vegas/7712-Coronado-Island-St-89139/home/29035729',
'https://www.redfin.com/NV/Las-Vegas/6621-Melodic-Ct-89139/home/29036616',
'https://www.redfin.com/NV/Las-Vegas/6258-Canebrake-Ct-89141/home/29038382',
'https://www.redfin.com/NV/Las-Vegas/1443-Corsica-Crest-Ct-89123/home/29038900',
'https://www.redfin.com/NV/Las-Vegas/4683-Ametrine-Ct-89139/home/29039047',
'https://www.redfin.com/NV/Henderson/10935-Scotch-Rose-St-89052/home/29040547',
'https://www.redfin.com/NV/Las-Vegas/10828-Rosabella-St-89141/home/29040643',
'https://www.redfin.com/NV/Las-Vegas/10576-Timber-Stand-St-89183/home/29040712',
'https://www.redfin.com/NV/Las-Vegas/8255-Cupertino-Heights-Way-89178/home/29056264',
'https://www.redfin.com/NV/Las-Vegas/8237-Campbell-Springs-Ave-89178/home/29056295',
'https://www.redfin.com/NV/Las-Vegas/9250-Valerie-Elaine-St-89139/home/29056416',
'https://www.redfin.com/NV/Las-Vegas/1841-Versante-Ave-89183/home/29057060',
'https://www.redfin.com/NV/Las-Vegas/7255-W-Sunset-Rd-89113/unit-1082/home/29065738',
'https://www.redfin.com/NV/Las-Vegas/9480-Swiftwater-Ct-89178/home/29068279',
'https://www.redfin.com/NV/Las-Vegas/2300-E-Silverado-Ranch-Blvd-89183/unit-1077/home/29070040',
'https://www.redfin.com/NV/Las-Vegas/2300-E-Silverado-Ranch-Blvd-89183/unit-2086/home/29070059',
'https://www.redfin.com/NV/Las-Vegas/555-E-Silverado-Ranch-Blvd-89183/unit-2037/home/29070177',
'https://www.redfin.com/NV/Las-Vegas/555-E-Silverado-Ranch-Blvd-89183/unit-2161/home/29070947',
'https://www.redfin.com/NV/Las-Vegas/9000-Las-Vegas-Blvd-S-89123/unit-1046/home/29184219',
'https://www.redfin.com/NV/Las-Vegas/9000-Las-Vegas-Blvd-S-89123/unit-1220/home/29184412',
'https://www.redfin.com/NV/Henderson/950-Seven-Hills-Dr-89052/unit-1016/home/29185327',
'https://www.redfin.com/NV/Henderson/950-Seven-Hills-Dr-89052/unit-1122/home/29185368',
'https://www.redfin.com/NV/Las-Vegas/9723-Skiers-Chalet-Ct-89178/home/29292506',
'https://www.redfin.com/NV/Las-Vegas/8054-Egypt-Meadows-Ave-89178/home/29292826',
'https://www.redfin.com/NV/Las-Vegas/9332-Vital-Crest-St-89123/home/29294891',
'https://www.redfin.com/NV/Henderson/1171-Adriana-Bend-Ct-89052/home/29296514',
'https://www.redfin.com/NV/Henderson/950-Seven-Hills-Dr-89052/unit-2522/home/29296966',
'https://www.redfin.com/NV/Las-Vegas/16-Harbor-Pines-Ct-89183/home/29303463',
'https://www.redfin.com/NV/Las-Vegas/11562-Elcadore-St-89183/home/29323046',
'https://www.redfin.com/NV/Henderson/251-S-Green-Valley-Pkwy-89012/unit-421/home/29450197',
'https://www.redfin.com/NV/Las-Vegas/7938-Homer-Ct-89139/home/29489118',
'https://www.redfin.com/NV/Las-Vegas/5708-Alington-Bend-Dr-89139/home/29490287',
'https://www.redfin.com/NV/Las-Vegas/10297-Burwood-St-89178/home/29490784',
'https://www.redfin.com/NV/Las-Vegas/481-Las-Alturas-Ave-89123/home/29493104',
'https://www.redfin.com/NV/Las-Vegas/68-E-Serene-Ave-89123/unit-416/home/29493777',
'https://www.redfin.com/NV/Las-Vegas/2405-W-Serene-Ave-89123/unit-735/home/29494050',
'https://www.redfin.com/NV/Las-Vegas/3163-Varenna-Ridge-Ave-89141/home/29495923',
'https://www.redfin.com/NV/Las-Vegas/259-Morning-Crest-Ave-89183/home/29499739',
'https://www.redfin.com/NV/Las-Vegas/9365-Lindell-Rd-89139/home/29591874',
'https://www.redfin.com/NV/Las-Vegas/6869-Tamarus-St-89119/unit-103/home/29592313',
'https://www.redfin.com/NV/Las-Vegas/1312-Finale-Ln-89119/home/29592353',
'https://www.redfin.com/NV/Las-Vegas/7218-Nordic-Lights-Dr-89119/home/29594180',
'https://www.redfin.com/NV/Las-Vegas/7123-Gunslinger-St-89119/home/29595084',
'https://www.redfin.com/NV/Las-Vegas/3345-W-Arby-Ave-89118/home/29595455',
'https://www.redfin.com/NV/Las-Vegas/4540-W-Pamalyn-Ave-89118/home/29595893',
'https://www.redfin.com/NV/Las-Vegas/4730-W-Robindale-Rd-89139/home/29596596',
'https://www.redfin.com/NV/Las-Vegas/3423-Beam-Dr-89139/home/29596986',
'https://www.redfin.com/NV/Las-Vegas/8025-Rancho-Destino-Rd-89123/home/29599387',
'https://www.redfin.com/NV/Las-Vegas/654-Ravenglen-Dr-89123/home/29607371',
'https://www.redfin.com/NV/Las-Vegas/354-Chaplin-Cove-Ave-89183/home/29609232',
'https://www.redfin.com/NV/Henderson/1168-Via-Piazza-89052/home/29610278',
'https://www.redfin.com/NV/Las-Vegas/932-Elysian-Dr-89123/home/29611604',
'https://www.redfin.com/NV/Las-Vegas/1170-Del-Lilly-Ln-89123/home/29611683',
'https://www.redfin.com/NV/Las-Vegas/7369-Cisco-Ln-89123/home/29611786',
'https://www.redfin.com/NV/Las-Vegas/917-Celebration-Dr-89123/home/29612750',
'https://www.redfin.com/NV/Henderson/2827-Via-Stella-St-89074/home/29614878',
'https://www.redfin.com/NV/Henderson/2764-Carnation-Ln-89074/home/29616151',
'https://www.redfin.com/NV/Henderson/219-Silado-Ct-89074/home/29617363',
'https://www.redfin.com/NV/Henderson/207-Chaco-Canyon-Dr-89074/home/29617554',
'https://www.redfin.com/NV/Henderson/141-Cologne-Ct-89074/home/29618029',
'https://www.redfin.com/NV/Henderson/2925-Wigwam-Pkwy-89074/unit-1321/home/29619183',
'https://www.redfin.com/NV/Henderson/2837-Yukon-Trail-Dr-89074/home/29619194',
'https://www.redfin.com/NV/Henderson/2925-Wigwam-Pkwy-89074/unit-1812/home/29619251',
'https://www.redfin.com/NV/Henderson/2925-Wigwam-Pkwy-89074/unit-223/home/29619515',
'https://www.redfin.com/NV/Henderson/103-Tapatio-St-89074/home/29620619',
'https://www.redfin.com/NV/Henderson/134-Coventry-Dr-89074/home/29620703',
'https://www.redfin.com/NV/Henderson/144-Coventry-Dr-89074/home/29620778',
'https://www.redfin.com/NV/Henderson/61-Tidwell-Ln-89074/home/29620873',
'https://www.redfin.com/NV/Las-Vegas/8863-Haviland-Rd-89123/home/29624528',
'https://www.redfin.com/NV/Las-Vegas/8845-Wallaby-Ln-89123/home/29624680',
'https://www.redfin.com/NV/Las-Vegas/832-Clear-Diamond-Ave-89123/home/29625543',
'https://www.redfin.com/NV/Las-Vegas/1129-Country-Shadows-Way-89123/home/29625690',
'https://www.redfin.com/NV/Las-Vegas/361-Cascade-Mist-Ave-89123/home/29629380',
'https://www.redfin.com/NV/Las-Vegas/9570-Soloshine-St-89123/home/29631068',
'https://www.redfin.com/NV/Las-Vegas/1025-Clover-Tip-Ct-89123/home/29633008',
'https://www.redfin.com/NV/Las-Vegas/2196-Madica-Ave-89123/home/29634934',
'https://www.redfin.com/NV/Henderson/8959-Sierra-Palms-Way-89074/home/29635474',
'https://www.redfin.com/NV/Henderson/502-Escoto-Pl-89052/home/29636717',
'https://www.redfin.com/NV/Las-Vegas/10665-Gilespie-St-89183/home/29640260',
'https://www.redfin.com/NV/Henderson/3158-Castle-Canyon-Ave-89052/home/29641192',
'https://www.redfin.com/NV/Henderson/3007-Scenic-Valley-Way-89052/home/29641291',
'https://www.redfin.com/NV/Henderson/3038-Scenic-Valley-Way-89052/home/29641312',
'https://www.redfin.com/NV/Henderson/1128-Via-Toscana-89052/home/29641699',
'https://www.redfin.com/NV/Henderson/2745-Middleburgh-Ct-89052/home/29641749',
'https://www.redfin.com/NV/Henderson/2427-Pickwick-Dr-89014/home/29647257',
'https://www.redfin.com/NV/Henderson/15-Quail-Hollow-Dr-89014/home/29651414',
'https://www.redfin.com/NV/Henderson/2356-Aqua-Vista-Ave-89014/home/29653105',
'https://www.redfin.com/NV/Henderson/364-Heath-Ct-89074/home/29654275',
'https://www.redfin.com/NV/Henderson/2532-El-Paso-Grande-Ave-89074/home/29655272',
'https://www.redfin.com/NV/Henderson/270-El-Camino-Verde-St-89074/home/29655410',
'https://www.redfin.com/NV/Henderson/264-El-Camino-Verde-St-89074/home/29655417',
'https://www.redfin.com/NV/Henderson/213-Malcolm-St-89074/home/29656365',
'https://www.redfin.com/NV/Henderson/2239-Le-Conte-Ave-89074/home/29657167',
'https://www.redfin.com/NV/Henderson/314-Pissarro-Pl-89074/home/29657561',
'https://www.redfin.com/NV/Henderson/2044-Waverly-Cir-89014/home/29657599',
'https://www.redfin.com/NV/Henderson/240-Windsong-Dr-89074/home/29658757',
'https://www.redfin.com/NV/Henderson/2379-Cliffwood-Dr-89074/home/29677069',
'https://www.redfin.com/NV/Henderson/193-Renville-Ct-89074/home/29677092',
'https://www.redfin.com/NV/Henderson/2484-Ram-Crossing-Way-89074/home/29677463',
'https://www.redfin.com/NV/Henderson/2503-Hacker-Dr-89074/home/29678293',
'https://www.redfin.com/NV/Henderson/2325-Windmill-Pkwy-89074/unit-912/home/29679265',
'https://www.redfin.com/NV/Henderson/216-Ultra-Dr-89074/home/29679343',
'https://www.redfin.com/NV/Henderson/2251-Wigwam-Pkwy-89074/unit-926/home/29679594',
'https://www.redfin.com/NV/Henderson/2251-Wigwam-Pkwy-89074/unit-1915/home/29679774',
'https://www.redfin.com/NV/Henderson/2455-Avenida-Cataluna-89074/home/29680533',
'https://www.redfin.com/NV/Henderson/78-Fortuna-Ct-89074/home/29680807',
'https://www.redfin.com/NV/Henderson/75-Motu-Ct-89074/home/29680825',
'https://www.redfin.com/NV/Henderson/16-Rue-de-Parc-89074/home/29681563',
'https://www.redfin.com/NV/Henderson/2267-Nashville-Ave-89052/home/29682267',
'https://www.redfin.com/NV/Henderson/25-Lantern-Glow-Cir-89074/home/29683006',
'https://www.redfin.com/NV/Henderson/2146-High-Mesa-Dr-89012/home/29692633',
'https://www.redfin.com/NV/Henderson/557-Carmel-Valley-St-89012/home/29693354',
'https://www.redfin.com/NV/Henderson/569-Fox-Links-St-89012/home/29693364',
'https://www.redfin.com/NV/Henderson/2526-Silver-Beach-Dr-89052/home/29696238',
'https://www.redfin.com/NV/Henderson/819-Adagio-St-89052/home/29697137',
'https://www.redfin.com/NV/Henderson/2578-Anchorgate-Dr-89052/home/29697154',
'https://www.redfin.com/NV/Henderson/2383-Falsetto-Ave-89052/home/29737279',
'https://www.redfin.com/NV/Henderson/2280-Lyrical-Rd-89052/home/29739606',
'https://www.redfin.com/NV/Henderson/1526-Villa-Rica-Dr-89052/home/29740897',
'https://www.redfin.com/NV/Henderson/1322-Imperia-Dr-89052/home/29740985',
'https://www.redfin.com/NV/Henderson/1376-European-Dr-89052/home/29742012',
'https://www.redfin.com/NV/Las-Vegas/8123-Amy-Springs-St-89113/home/29769023',
'https://www.redfin.com/NV/Las-Vegas/7295-Charreado-Ct-89179/home/29769697',
'https://www.redfin.com/NV/Las-Vegas/7190-La-Cadena-Ave-89179/home/29794959',
'https://www.redfin.com/NV/Las-Vegas/8255-Las-Vegas-Blvd-S-89123/unit-911/home/29800270',
'https://www.redfin.com/NV/Las-Vegas/2055-Dragonslayer-Ave-89183/home/29825045',
'https://www.redfin.com/NV/Las-Vegas/7633-Calm-Passage-Ct-89139/home/29834243',
'https://www.redfin.com/NV/Las-Vegas/17-Belle-La-Blanc-Ave-89123/home/29835417',
'https://www.redfin.com/NV/Henderson/691-Great-Dane-Ct-89052/home/29836043',
'https://www.redfin.com/NV/Henderson/2737-Bass-Lake-St-89052/home/29836999',
'https://www.redfin.com/NV/Henderson/2307-Silver-Crew-Pass-89052/home/29839918',
'https://www.redfin.com/NV/Henderson/1475-Cilento-Ct-89052/home/29840861',
'https://www.redfin.com/NV/Las-Vegas/10932-Sospel-Pl-89141/home/29848484',
'https://www.redfin.com/NV/Las-Vegas/4348-Val-Dechiana-Ave-89141/home/29849591',
'https://www.redfin.com/NV/Henderson/2662-Sunday-Grace-Dr-89052/home/29875857',
'https://www.redfin.com/NV/Las-Vegas/10010-Swimming-Hole-St-89183/home/29876812',
'https://www.redfin.com/NV/Henderson/700-Carnegie-St-89052/unit-913/home/29882606',
'https://www.redfin.com/NV/Las-Vegas/10621-Bardilino-St-89141/home/29889070',
'https://www.redfin.com/NV/Las-Vegas/595-Campus-Oaks-Ct-89183/home/29890978',
'https://www.redfin.com/NV/Las-Vegas/10502-Bonchester-Hill-St-89141/home/29900348',
'https://www.redfin.com/NV/Henderson/700-Carnegie-St-89052/unit-4712/home/29903610',
'https://www.redfin.com/NV/Henderson/1240-Sonatina-Dr-89052/home/29904218',
'https://www.redfin.com/NV/Las-Vegas/5557-San-Florentine-Ave-89141/home/29909868',
'https://www.redfin.com/NV/Henderson/2146-Handel-Ave-89052/home/29913686',
'https://www.redfin.com/NV/Las-Vegas/11011-Santorini-Dr-89141/home/29918661',
'https://www.redfin.com/NV/Las-Vegas/5773-Lazy-Days-Ct-89141/home/29920834',
'https://www.redfin.com/NV/Las-Vegas/5137-Numidia-Ave-89141/home/29940062',
'https://www.redfin.com/NV/Las-Vegas/10779-Barnard-Bee-Ct-89183/home/29942835',
'https://www.redfin.com/NV/Henderson/830-Carnegie-St-89052/unit-1222/home/29943773',
'https://www.redfin.com/NV/Henderson/2144-Paganini-Ave-89052/home/29945403',
'https://www.redfin.com/NV/Henderson/2144-Volante-Cir-89052/home/29946319',
'https://www.redfin.com/NV/Las-Vegas/9131-Marlia-St-89123/home/29950193',
'https://www.redfin.com/NV/Las-Vegas/7123-Hickory-Post-Ave-89179/home/30182713',
'https://www.redfin.com/NV/Las-Vegas/5695-Dorian-Black-Ave-89139/home/30475645',
'https://www.redfin.com/NV/Las-Vegas/7086-Positano-Hill-Ave-89178/home/30476247',
'https://www.redfin.com/NV/Las-Vegas/7596-Brisk-Ocean-Ave-89178/home/39657329',
'https://www.redfin.com/NV/Henderson/3664-Via-Vicchio-89052/home/45356842',
'https://www.redfin.com/NV/Henderson/1193-Via-Monticano-89052/home/52628249',
'https://www.redfin.com/NV/Las-Vegas/6728-Philharmonic-Ave-89139/home/65553205',
'https://www.redfin.com/NV/Las-Vegas/7588-Brisk-Ocean-Ave-89178/home/65553257',
'https://www.redfin.com/NV/Las-Vegas/5441-Ledgewood-Creek-Ave-89141/home/65557289',
'https://www.redfin.com/NV/Las-Vegas/6771-Medovina-Ct-89139/home/101992513',
'https://www.redfin.com/NV/Las-Vegas/9607-Gray-Cap-St-89123/home/105538903',
'https://www.redfin.com/NV/Las-Vegas/8175-Arville-St-89139/unit-305/home/108927532',
'https://www.redfin.com/NV/Las-Vegas/7283-Shannon-Ridge-Ct-89118/home/109871047',
'https://www.redfin.com/NV/Las-Vegas/3977-Jacob-Lake-Cir-89118/home/143026488',
'https://www.redfin.com/NV/Las-Vegas/11446-Quantum-Brooks-St-89183/home/145871212',
'https://www.redfin.com/NV/Las-Vegas/81-Kaku-Ridge-Way-89183/home/145871217',
'https://www.redfin.com/NV/Henderson/869-Bluebell-Brook-St-89052/home/146417769',
'https://www.redfin.com/NV/Las-Vegas/7125-Steele-Canyon-St-89118/home/165931045',
'https://www.redfin.com/NV/Henderson/2100-Havensight-Ln-89052/home/166035301',
'https://www.redfin.com/NV/Las-Vegas/3960-W-Torino-Ave-89139/home/166396644',
'https://www.redfin.com/NV/Las-Vegas/11296-Villa-Bellagio-Dr-89141/home/166408286',
'https://www.redfin.com/NV/Las-Vegas/8512-Baltimore-Heights-Ct-89113/home/167759790',
'https://www.redfin.com/NV/Henderson/3783-Canis-Minor-Ln-89052/unit-6206/home/170139492',
'https://www.redfin.com/NV/Henderson/1256-Jaya-Peak-Ct-89052/home/170976927',
'https://www.redfin.com/NV/Las-Vegas/6264-Dolosonte-ST-89148/unit-44/home/185368268',
'https://www.redfin.com/NV/Las-Vegas/10562-Pueblo-Springs-St-89183/home/28993598',
'https://www.redfin.com/NV/Las-Vegas/9106-Mount-Wilson-St-89113/home/29321134',
'https://www.redfin.com/NV/Las-Vegas/9137-Hunting-Arrow-St-89123/home/29629667',
'https://www.redfin.com/NV/Las-Vegas/1438-Silver-Knoll-Ave-89123/home/29633863',
'https://www.redfin.com/NV/Las-Vegas/2186-Lipari-Ct-89123/home/29634124',
'https://www.redfin.com/NV/Henderson/215-Animated-Ct-89052/home/29942638',
'https://www.redfin.com/NV/Las-Vegas/9920-Mills-Reef-Ct-89141/home/39658664',
'https://www.redfin.com/NV/Las-Vegas/6350-White-Heron-Ct-89139/home/65556076',
'https://www.redfin.com/NV/Las-Vegas/7610-W-Arby-Ave-89113/home/166316659',
'https://www.redfin.com/NV/Las-Vegas/4390-W-Cactus-Ave-89141/home/170139122',
'https://www.redfin.com/NV/Las-Vegas/8318-Waverton-Ct-89113/home/183993368',
'https://www.redfin.com/NV/Henderson/2970-W-Horizon-Ridge-Pkwy-89052/unit-C/home/184802741',
'https://www.redfin.com/NV/Henderson/1475-Cilento-Ct-89052/home/186909961',
'https://www.redfin.com/NV/Henderson/167-White-Butte-St-89012/home/28967797',
'https://www.redfin.com/NV/Henderson/522-Kristin-Ln-89011/home/28969942',
'https://www.redfin.com/NV/Henderson/1561-Buffalo-Brubaker-Ln-89002/home/28974050',
'https://www.redfin.com/NV/Henderson/697-Blooming-Sage-Ct-89015/home/28979442',
'https://www.redfin.com/NV/Henderson/30-Strada-Di-Villaggio-89011/unit-349/home/29028316',
'https://www.redfin.com/NV/Henderson/1093-Pleasing-Plateau-St-89002/home/29041933',
'https://www.redfin.com/NV/Henderson/515-Moses-Lake-Ct-89002/home/29042156',
'https://www.redfin.com/NV/Henderson/995-Klamath-River-Ave-89002/home/29042173',
'https://www.redfin.com/NV/Henderson/2104-Savannah-River-St-89044/home/29042661',
'https://www.redfin.com/NV/Henderson/2310-Sandstone-Cliffs-Dr-89044/home/29042691',
'https://www.redfin.com/NV/Henderson/1979-Sunset-Bend-Dr-89014/home/29056832',
'https://www.redfin.com/NV/Henderson/1130-Yellow-Orchid-St-89002/home/29057270',
'https://www.redfin.com/NV/Las-Vegas/5263-Tipper-Ave-89122/home/29282799',
'https://www.redfin.com/NV/Las-Vegas/5710-E-Tropicana-Ave-89122/unit-1053/home/29431906',
'https://www.redfin.com/NV/Las-Vegas/5094-Mascaro-Dr-89122/home/29434163',
'https://www.redfin.com/NV/Las-Vegas/5215-Suzy-Saly-Pl-89122/home/29435336',
'https://www.redfin.com/NV/Las-Vegas/6312-Sabora-Ave-89122/home/29435995',
'https://www.redfin.com/NV/Las-Vegas/5336-Sunbury-Ave-89122/home/29436707',
'https://www.redfin.com/NV/Las-Vegas/5448-Wellesley-Dr-89122/home/29437542',
'https://www.redfin.com/NV/Henderson/721-Rising-Brook-Dr-89011/home/29447695',
'https://www.redfin.com/NV/Henderson/728-Rising-Brook-Dr-89011/home/29447746',
'https://www.redfin.com/NV/Henderson/621-Moontide-Ct-89011/home/29447940',
'https://www.redfin.com/NV/Henderson/45-Maleena-Mesa-St-89074/unit-925/home/29610177',
'https://www.redfin.com/NV/Henderson/45-Maleena-Mesa-St-89074/unit-1623/home/29610936',
'https://www.redfin.com/NV/Henderson/1929-Bearden-Ave-89011/home/29641790',
'https://www.redfin.com/NV/Henderson/554-Kristin-Ln-89011/home/29643014',
'https://www.redfin.com/NV/Henderson/1719-Empire-Mine-Dr-89014/home/29644495',
'https://www.redfin.com/NV/Henderson/400-Raindance-Dr-89014/home/29645248',
'https://www.redfin.com/NV/Henderson/727-Apple-Tree-Ct-89014/home/29649625',
'https://www.redfin.com/NV/Henderson/604-Cervantes-Dr-89014/home/29650086',
'https://www.redfin.com/NV/Henderson/2711-Osborne-Ln-89014/home/29650731',
'https://www.redfin.com/NV/Henderson/340-Fuente-Dr-89014/home/29660596',
'https://www.redfin.com/NV/Henderson/384-Seine-Way-89014/home/29662776',
'https://www.redfin.com/NV/Henderson/1709-Sequoia-Dr-89014/home/29664002',
'https://www.redfin.com/NV/Henderson/321-Perry-Ellis-Dr-89014/home/29665676',
'https://www.redfin.com/NV/Henderson/304-Saint-Augustine-Ln-89014/home/29666658',
'https://www.redfin.com/NV/Henderson/1194-Golden-Spike-Ct-89014/home/29668070',
'https://www.redfin.com/NV/Henderson/1149-Light-Sky-Ave-89074/home/29668245',
'https://www.redfin.com/NV/Henderson/237-Collindale-St-89074/home/29668309',
'https://www.redfin.com/NV/Henderson/196-Boothbay-St-89074/home/29669216',
'https://www.redfin.com/NV/Henderson/277-Ventana-Heights-St-89074/home/29669290',
'https://www.redfin.com/NV/Henderson/1134-Evening-Canyon-Ave-89014/home/29669453',
'https://www.redfin.com/NV/Henderson/205-Carson-Way-89015/home/29670288',
'https://www.redfin.com/NV/Henderson/245-Carson-Way-89015/home/29670485',
'https://www.redfin.com/NV/Henderson/11-Desert-Dawn-Ln-89074/home/29673696',
'https://www.redfin.com/NV/Henderson/194-Webster-Way-89074/home/29674736',
'https://www.redfin.com/NV/Henderson/2002-Spruce-Brook-Dr-89074/home/29677405',
'https://www.redfin.com/NV/Henderson/1951-Thunder-Ridge-Cir-89012/home/29685482',
'https://www.redfin.com/NV/Henderson/1722-Summerwood-Cir-89012/home/29686023',
'https://www.redfin.com/NV/Henderson/1828-Thunder-Mountain-Dr-89012/home/29686318',
'https://www.redfin.com/NV/Henderson/428-Wright-Way-89015/home/29689939',
'https://www.redfin.com/NV/Henderson/410-Daffodil-Dr-89015/home/29690912',
'https://www.redfin.com/NV/Henderson/81-Zimmerman-St-89002/home/29691586',
'https://www.redfin.com/NV/Henderson/361-Island-Reef-Ave-89012/home/29691631',
'https://www.redfin.com/NV/Henderson/2122-High-Mesa-Dr-89012/home/29692610',
'https://www.redfin.com/NV/Henderson/480-Eagle-Vista-Dr-89012/home/29694141',
'https://www.redfin.com/NV/Henderson/1816-Cypress-Mesa-Dr-89012/home/29699072',
'https://www.redfin.com/NV/Henderson/221-Fir-St-89015/home/29702926',
'https://www.redfin.com/NV/Henderson/1015-Brookside-Ct-89015/home/29703073',
'https://www.redfin.com/NV/Henderson/1003-Center-St-89015/home/29703118',
'https://www.redfin.com/NV/Henderson/431-Bell-Ave-89015/home/29703316',
'https://www.redfin.com/NV/Henderson/265-Dahlia-St-89015/home/29704209',
'https://www.redfin.com/NV/Henderson/211-Oleander-Pl-89015/home/29704350',
'https://www.redfin.com/NV/Henderson/220-Pretty-Sunset-Ter-89015/home/29704652',
'https://www.redfin.com/NV/Henderson/814-Anchor-Dr-89015/home/29704723',
'https://www.redfin.com/NV/Henderson/173-Skytop-Dr-89015/home/29705536',
'https://www.redfin.com/NV/Henderson/111-Maple-St-89015/home/29706518',
'https://www.redfin.com/NV/Henderson/601-Cabrillo-Cir-89015/unit-317/home/29706545',
'https://www.redfin.com/NV/Henderson/241-Queen-Marie-Ct-89015/home/29707404',
'https://www.redfin.com/NV/Henderson/425-McKenzie-Dr-89015/home/29707667',
'https://www.redfin.com/NV/Henderson/103-E-Pacific-Ave-89015/home/29709409',
'https://www.redfin.com/NV/Henderson/246-Tungsten-St-89015/home/29709785',
'https://www.redfin.com/NV/Henderson/203-E-Van-Wagenen-St-89015/home/29711049',
'https://www.redfin.com/NV/Henderson/315-E-Van-Wagenen-St-89015/home/29711096',
'https://www.redfin.com/NV/Henderson/984-Broken-Reed-Ct-89015/home/29714128',
'https://www.redfin.com/NV/Henderson/989-Trackers-Glen-Ave-89015/home/29714404',
'https://www.redfin.com/NV/Henderson/2042-Buckeye-Reef-St-89002/home/29716825',
'https://www.redfin.com/NV/Henderson/1581-Equestrian-Dr-89002/home/29717557',
'https://www.redfin.com/NV/Henderson/906-Big-Horn-St-89002/home/29719027',
'https://www.redfin.com/NV/Henderson/406-Scenic-Dr-89002/home/29719844',
'https://www.redfin.com/NV/Henderson/723-Strawberry-Pl-89002/home/29721948',
'https://www.redfin.com/NV/Henderson/124-Mint-Orchard-Dr-89002/home/29722416',
'https://www.redfin.com/NV/Henderson/113-Tamarack-Dr-89002/home/29722493',
'https://www.redfin.com/NV/Henderson/825-Cypress-Pines-Way-89002/home/29724520',
'https://www.redfin.com/NV/Henderson/847-Woodtack-Cove-Way-89002/home/29725328',
'https://www.redfin.com/NV/Boulder-City/1310-Marita-Dr-89005/home/29732402',
'https://www.redfin.com/NV/Boulder-City/768-Christina-Dr-89005/home/29733571',
'https://www.redfin.com/NV/Boulder-City/1504-Irene-Dr-89005/home/29735883',
'https://www.redfin.com/NV/Boulder-City/889-Shirley-Ln-89005/home/29736900',
'https://www.redfin.com/NV/Henderson/231-W-Horizon-Ridge-Pkwy-89012/unit-211/home/29772328',
'https://www.redfin.com/NV/Henderson/50-Aura-de-Blanco-St-89074/unit-17104/home/29772579',
'https://www.redfin.com/NV/Henderson/560-Carmel-Mesa-Dr-89012/home/29838580',
'https://www.redfin.com/NV/Henderson/201-Crown-Imperial-St-89074/home/29880007',
'https://www.redfin.com/NV/Henderson/2556-Forest-City-Dr-89052/home/29886070',
'https://www.redfin.com/NV/Henderson/2008-Colvin-Run-Dr-89052/home/29888844',
'https://www.redfin.com/NV/Henderson/6412-Saddle-Up-Ave-89011/unit-102/home/29894805',
'https://www.redfin.com/NV/Henderson/16-Wade-Hampton-Trl-89052/home/29905296',
'https://www.redfin.com/NV/Henderson/3019-Stratmoor-Hills-Ave-89052/home/29905347',
'https://www.redfin.com/NV/Henderson/91-Stock-Option-St-89074/unit-2/home/29915734',
'https://www.redfin.com/NV/Henderson/1285-Large-Cap-Dr-89074/unit-2/home/29942565',
'https://www.redfin.com/NV/Henderson/728-Tossa-de-Mar-Ave-89002/home/29943660',
'https://www.redfin.com/NV/Henderson/2092-Mountain-City-St-89052/home/29947255',
'https://www.redfin.com/NV/Boulder-City/375-Crystal-Ct-89005/home/33538666',
'https://www.redfin.com/NV/Henderson/789-Calamus-Palm-Pl-89011/home/52623922',
'https://www.redfin.com/NV/Henderson/1309-Jewelstone-Cir-89012/home/60188060',
'https://www.redfin.com/NV/Henderson/1977-Via-Firenze-89044/home/65556563',
'https://www.redfin.com/NV/Henderson/50-Aura-de-Blanco-St-89074/unit-5201/home/109621648',
'https://www.redfin.com/NV/Henderson/1109-Via-Della-Costrella-89011/home/109633497',
'https://www.redfin.com/NV/Henderson/701-Tidal-Flats-St-89002/home/113140376',
'https://www.redfin.com/NV/Henderson/1525-Spiced-Wine-Ave-89074/unit-16104/home/143400805',
'https://www.redfin.com/NV/Las-Vegas/6435-Point-Isabel-Way-89122/home/145870763',
'https://www.redfin.com/NV/Henderson/257-Via-Morelli-89011/home/146416200',
'https://www.redfin.com/NV/Henderson/1072-Upper-Springs-St-89002/home/147876643',
'https://www.redfin.com/NV/Henderson/338-Espressivo-St-89011/home/161754406',
'https://www.redfin.com/NV/Henderson/378-Badinerie-St-89011/home/161755161',
'https://www.redfin.com/NV/Henderson/4-Cloudloft-Ct-89012/home/166426816',
'https://www.redfin.com/NV/Henderson/211-E-Kimberly-Dr-89015/home/166566234',
'https://www.redfin.com/NV/Henderson/403-Andy-Wheeler-Dr-89011/home/167787119',
'https://www.redfin.com/NV/Boulder-City/1535-Bryce-Canyon-St-89005/home/167795877',
'https://www.redfin.com/NV/Henderson/624-Cultures-St-89011/home/169241339',
'https://www.redfin.com/NV/Henderson/965-Nevada-State-Dr-89002/unit-21102/home/169242844',
'https://www.redfin.com/NV/Henderson/89-Lake-Oasis-St-89011/home/171301906',
'https://www.redfin.com/NV/Henderson/932-E-Sunset-Rd-89011/home/171302356',
'https://www.redfin.com/NV/Henderson/361-Seine-Way-89014/unit-1311/home/172345414',
'https://www.redfin.com/NV/Henderson/731-Cottonwood-Hill-Pl-89011/home/175528274',
'https://www.redfin.com/NV/Henderson/729-Cottonwood-Hill-Pl-89011/home/175528275',
'https://www.redfin.com/NV/Henderson/307-Bay-Village-Pl-89011/home/177309564',
'https://www.redfin.com/NV/Henderson/57-Tatahatso-Point-St-89011/home/177327882',
'https://www.redfin.com/NV/Henderson/377-Old-Erie-Pl-89011/home/184007626',
'https://www.redfin.com/NV/Henderson/48-Cadence-Crest-Ave-89011/home/185121364',
'https://www.redfin.com/NV/Henderson/879-Mary-Baer-Way-89011/home/185121370',
'https://www.redfin.com/NV/Henderson/227-Wewatta-AVE-Unknown/home/185575654',
'https://www.redfin.com/NV/Henderson/384-Winter-Dove-ST-89011/unit-298/home/186023685',
'https://www.redfin.com/NV/Henderson/388-Blanca-Springs-Dr-89014/unit-341/home/186133092',
'https://www.redfin.com/NV/Henderson/479-Waterfall-Cove-Ct-89002/home/185775167',
'https://www.redfin.com/NV/Henderson/342-Larksong-AVE-89011/unit-81/home/186090400',
'https://www.redfin.com/NV/Henderson/344-Larksong-AVE-89011/unit-82/home/186138150',
'https://www.redfin.com/NV/Henderson/340-Larksong-AVE-89011/unit-80/home/186139372',
'https://www.redfin.com/NV/Henderson/885-Almandine-Pl-89011/unit-112/home/186161032',
'https://www.redfin.com/NV/Henderson/889-Almandine-Pl-89011/unit-113/home/186161049',
'https://www.redfin.com/NV/Henderson/905-Almandine-Pl-89011/unit-117/home/186161050',
'https://www.redfin.com/NV/Henderson/348-Nesting-Dove-ST-89011/unit-289/home/186200010',
'https://www.redfin.com/NV/Henderson/495-Blue-Ridge-CV-89002/unit-63/home/186205322',
'https://www.redfin.com/NV/Henderson/334-Larksong-AVE-89011/unit-77/home/186207851',
'https://www.redfin.com/NV/Henderson/338-Larksong-AVE-89011/unit-79/home/186208477',
'https://www.redfin.com/NV/Henderson/667-Anemone-Ln-89015/unit-249/home/186215558',
'https://www.redfin.com/NV/Henderson/1187-King-St-89011/unit-37/home/186274557',
'https://www.redfin.com/NV/Henderson/336-Larksong-AVE-89011/unit-78/home/186274617',
'https://www.redfin.com/NV/Henderson/1185-King-St-89011/unit-38/home/186298924',
'https://www.redfin.com/NV/Henderson/333-Seine-Way-89014/unit-1722/home/186318427',
'https://www.redfin.com/NV/Henderson/661-Anemone-Ln-89015/unit-252/home/186335917',
'https://www.redfin.com/NV/Henderson/1181-King-St-89011/unit-40/home/186373631',
'https://www.redfin.com/NV/Las-Vegas/4555-Winter-Place-St-89122/home/29014966',
'https://www.redfin.com/NV/Henderson/717-Oak-Field-Ln-89011/home/29448320',
'https://www.redfin.com/NV/Henderson/416-Bottle-Brush-Way-89015/home/29690847',
'https://www.redfin.com/NV/Boulder-City/667-Avenue-F-89005/home/29732587',
'https://www.redfin.com/NV/Henderson/856-Chameleon-Star-Ave-89015/home/29943837',
'https://www.redfin.com/NV/Henderson/1071-Fish-Pond-Ave-89014/home/64091870',
'https://www.redfin.com/NV/North-Las-Vegas/5816-Michael-Dean-St-89081/home/28964405',
'https://www.redfin.com/NV/North-Las-Vegas/3533-Rubio-Sun-Ave-89081/home/28968439',
'https://www.redfin.com/NV/North-Las-Vegas/323-River-Glider-Ave-89084/home/28969589',
'https://www.redfin.com/NV/North-Las-Vegas/4604-Checker-Way-89031/home/28970257',
'https://www.redfin.com/NV/North-Las-Vegas/1412-Iris-Kelly-Ave-89081/home/28971917',
'https://www.redfin.com/NV/North-Las-Vegas/5744-Dawson-Creek-St-89031/home/28972356',
'https://www.redfin.com/NV/North-Las-Vegas/4643-Mountain-Snow-St-89031/home/28972640',
'https://www.redfin.com/NV/North-Las-Vegas/5855-Valley-Dr-89031/unit-2119/home/28974895',
'https://www.redfin.com/NV/North-Las-Vegas/633-Claxton-Ave-89084/home/28975847',
'https://www.redfin.com/NV/North-Las-Vegas/6516-Copper-Smith-Ct-89084/home/28977295',
'https://www.redfin.com/NV/North-Las-Vegas/3012-Fern-Crest-Ave-89031/home/28977338',
'https://www.redfin.com/NV/North-Las-Vegas/3208-Tanagrine-Dr-89084/home/28980511',
'https://www.redfin.com/NV/North-Las-Vegas/3033-Dowitcher-Ave-89084/home/28980619',
'https://www.redfin.com/NV/North-Las-Vegas/3516-Glorious-Iris-Pl-89084/unit-1/home/28981019',
'https://www.redfin.com/NV/North-Las-Vegas/6625-Lavender-Lilly-Ln-89084/unit-1/home/28981029',
'https://www.redfin.com/NV/North-Las-Vegas/2128-Saddlebill-Ct-89084/home/28981184',
'https://www.redfin.com/NV/North-Las-Vegas/6740-Sand-Swallow-St-89084/home/28981223',
'https://www.redfin.com/NV/North-Las-Vegas/6624-Tumbler-St-89084/home/28981944',
'https://www.redfin.com/NV/North-Las-Vegas/2509-Silvereye-Dr-89084/home/28981947',
'https://www.redfin.com/NV/North-Las-Vegas/4925-Celsion-Rock-St-89081/home/28982641',
'https://www.redfin.com/NV/North-Las-Vegas/4811-Donna-St-89081/home/28982658',
'https://www.redfin.com/NV/North-Las-Vegas/3441-Brolio-Valley-Ct-89032/home/28987404',
'https://www.redfin.com/NV/North-Las-Vegas/5913-Horsehair-Blanket-Dr-89081/home/28997329',
'https://www.redfin.com/NV/North-Las-Vegas/5245-Coral-Hills-St-89081/home/28998014',
'https://www.redfin.com/NV/North-Las-Vegas/1151-Sunny-Acres-Ave-89081/home/28998494',
'https://www.redfin.com/NV/North-Las-Vegas/3128-Gnatcatcher-Ave-89084/home/28999391',
'https://www.redfin.com/NV/North-Las-Vegas/3116-Gnatcatcher-Ave-89084/home/28999547',
'https://www.redfin.com/NV/North-Las-Vegas/6237-Olympic-Gold-St-89031/home/29001655',
'https://www.redfin.com/NV/Las-Vegas/4106-Pistachio-Nut-Ave-89115/home/29007135',
'https://www.redfin.com/NV/North-Las-Vegas/6016-Leaping-Foal-St-89081/home/29010427',
'https://www.redfin.com/NV/North-Las-Vegas/4604-Grey-Heron-Dr-89084/home/29010786',
'https://www.redfin.com/NV/North-Las-Vegas/6208-Darby-Creek-Ct-89081/home/29011450',
'https://www.redfin.com/NV/North-Las-Vegas/1238-Shades-End-Ave-89081/home/29013683',
'https://www.redfin.com/NV/North-Las-Vegas/3832-Alpine-Bypass-Ave-89081/home/29016184',
'https://www.redfin.com/NV/North-Las-Vegas/3613-Fledgling-Dr-89084/home/29016841',
'https://www.redfin.com/NV/North-Las-Vegas/6224-Sun-Seed-Ct-89081/home/29016935',
'https://www.redfin.com/NV/North-Las-Vegas/4444-Carrier-Dove-Ave-89084/home/29017322',
'https://www.redfin.com/NV/North-Las-Vegas/4124-Erinbird-Ave-89084/home/29017779',
'https://www.redfin.com/NV/North-Las-Vegas/4021-California-Condor-Ave-89084/home/29017800',
'https://www.redfin.com/NV/North-Las-Vegas/4605-White-Stork-Dr-89084/home/29017958',
'https://www.redfin.com/NV/North-Las-Vegas/7082-Seabirds-Pl-89084/home/29018200',
'https://www.redfin.com/NV/North-Las-Vegas/3320-Majestic-Sand-Ave-89031/home/29019316',
'https://www.redfin.com/NV/North-Las-Vegas/5841-Vista-Luna-St-89031/home/29019341',
'https://www.redfin.com/NV/North-Las-Vegas/4324-Hera-Temple-Ave-89031/home/29020124',
'https://www.redfin.com/NV/North-Las-Vegas/4117-Boston-Bell-Ct-89031/home/29020137',
'https://www.redfin.com/NV/North-Las-Vegas/113-Frad-Ave-89031/home/29020641',
'https://www.redfin.com/NV/Las-Vegas/5115-Thataboy-Ct-89130/home/29024580',
'https://www.redfin.com/NV/Las-Vegas/4555-Hearts-Desire-Ave-89115/home/29028084',
'https://www.redfin.com/NV/North-Las-Vegas/713-W-Colton-Ave-89032/home/29044051',
'https://www.redfin.com/NV/North-Las-Vegas/5909-Addy-Ln-89081/home/29048533',
'https://www.redfin.com/NV/North-Las-Vegas/3021-Red-Imp-Ave-89081/home/29049078',
'https://www.redfin.com/NV/North-Las-Vegas/6213-Capehart-Falls-St-89081/home/29049214',
'https://www.redfin.com/NV/North-Las-Vegas/6012-Valley-Flower-St-89081/home/29049437',
'https://www.redfin.com/NV/North-Las-Vegas/3129-Little-Crimson-Ave-89081/home/29049592',
'https://www.redfin.com/NV/North-Las-Vegas/5450-Tea-Leaf-St-89031/home/29049976',
'https://www.redfin.com/NV/North-Las-Vegas/1944-W-Hammer-Ln-89031/home/29050197',
'https://www.redfin.com/NV/Las-Vegas/6070-Burleson-Ranch-Rd-89131/home/29051414',
'https://www.redfin.com/NV/North-Las-Vegas/5721-Leonidas-St-89031/home/29059830',
'https://www.redfin.com/NV/North-Las-Vegas/1217-Checkmark-Ave-89032/home/29061929',
'https://www.redfin.com/NV/North-Las-Vegas/5828-Meadow-Dale-Dr-89031/home/29076973',
'https://www.redfin.com/NV/North-Las-Vegas/1704-Black-Hills-Way-89031/home/29077663',
'https://www.redfin.com/NV/North-Las-Vegas/1601-Heather-Ridge-Rd-89031/home/29078367',
'https://www.redfin.com/NV/North-Las-Vegas/908-Crescent-Moon-Dr-89031/home/29078861',
'https://www.redfin.com/NV/North-Las-Vegas/1227-Dover-Glen-Dr-89031/home/29078905',
'https://www.redfin.com/NV/North-Las-Vegas/5604-Indian-Ridge-Dr-89031/home/29079374',
'https://www.redfin.com/NV/North-Las-Vegas/1440-Walstone-Rd-89031/home/29079510',
'https://www.redfin.com/NV/North-Las-Vegas/2603-Sweet-Leilani-Ave-89031/home/29079609',
'https://www.redfin.com/NV/North-Las-Vegas/2123-Maui-Surf-Ave-89031/home/29080087',
'https://www.redfin.com/NV/North-Las-Vegas/2113-Port-Antonio-Ct-89031/home/29083142',
'https://www.redfin.com/NV/North-Las-Vegas/2108-Spanish-Town-Ave-89031/home/29083184',
'https://www.redfin.com/NV/North-Las-Vegas/4836-Abundance-St-89031/home/29083234',
'https://www.redfin.com/NV/North-Las-Vegas/5117-Sierra-Blanca-Ln-89031/home/29083741',
'https://www.redfin.com/NV/North-Las-Vegas/5119-El-Nino-Ln-89031/home/29083744',
'https://www.redfin.com/NV/North-Las-Vegas/1133-Turnstone-Ct-89031/home/29084617',
'https://www.redfin.com/NV/North-Las-Vegas/1125-Egret-Ct-89031/home/29084728',
'https://www.redfin.com/NV/North-Las-Vegas/1217-Stonesthrow-Cir-89031/home/29084754',
'https://www.redfin.com/NV/North-Las-Vegas/5539-Jose-Leon-St-89031/home/29085104',
'https://www.redfin.com/NV/North-Las-Vegas/5241-Jose-Ernesto-St-89031/home/29085326',
'https://www.redfin.com/NV/North-Las-Vegas/220-Breezy-Shore-Ave-89031/home/29086727',
'https://www.redfin.com/NV/Las-Vegas/6016-Morning-Creek-Ct-89130/home/29092774',
'https://www.redfin.com/NV/Las-Vegas/6320-Whispering-Meadow-Ct-89130/home/29093064',
'https://www.redfin.com/NV/Las-Vegas/6317-Winter-Valley-Ct-89130/home/29093066',
'https://www.redfin.com/NV/Las-Vegas/6016-Winter-Meadow-St-89130/home/29093120',
'https://www.redfin.com/NV/Las-Vegas/5916-Trumbull-St-89130/home/29093246',
'https://www.redfin.com/NV/Las-Vegas/6101-Shallow-Springs-St-89130/home/29093517',
'https://www.redfin.com/NV/Las-Vegas/5421-Singing-Hills-Dr-89130/home/29102028',
'https://www.redfin.com/NV/Las-Vegas/5317-Braemar-Dr-89130/home/29102324',
'https://www.redfin.com/NV/Las-Vegas/5533-Port-Barrington-Way-89130/home/29102467',
'https://www.redfin.com/NV/Las-Vegas/4937-Rancho-Bernardo-Way-89130/home/29102624',
'https://www.redfin.com/NV/Las-Vegas/5604-Ocean-Pines-Cir-89130/home/29102698',
'https://www.redfin.com/NV/Las-Vegas/5037-Cedar-Lawn-Way-89130/home/29102964',
'https://www.redfin.com/NV/Las-Vegas/4533-Mancilla-St-89130/home/29115744',
'https://www.redfin.com/NV/Las-Vegas/4504-Plano-Dr-89130/home/29115859',
'https://www.redfin.com/NV/Las-Vegas/5424-Del-Lago-Dr-89130/home/29116287',
'https://www.redfin.com/NV/Las-Vegas/4980-W-San-Miguel-Ave-89130/home/29116772',
'https://www.redfin.com/NV/Las-Vegas/5908-Duncan-Dr-89108/home/29132405',
'https://www.redfin.com/NV/Las-Vegas/5812-Robin-Ln-89108/home/29133318',
'https://www.redfin.com/NV/Las-Vegas/3205-Point-Lobos-Dr-89108/home/29134056',
'https://www.redfin.com/NV/Las-Vegas/3916-Wharton-St-89130/home/29134258',
'https://www.redfin.com/NV/Las-Vegas/5515-Sheila-Ave-89108/home/29134938',
'https://www.redfin.com/NV/Las-Vegas/5425-Geneive-Ln-89108/unit-104/home/29137142',
'https://www.redfin.com/NV/Las-Vegas/5401-Geneive-Ln-89108/unit-101/home/29137198',
'https://www.redfin.com/NV/Las-Vegas/2204-Obispo-Cir-89108/home/29172566',
'https://www.redfin.com/NV/Las-Vegas/5917-Smoke-Ranch-Rd-89108/unit-D/home/29174373',
'https://www.redfin.com/NV/Las-Vegas/1628-N-Jones-Blvd-89108/home/29177264',
'https://www.redfin.com/NV/Las-Vegas/5378-W-Lake-Mead-Blvd-89108/home/29177321',
'https://www.redfin.com/NV/North-Las-Vegas/4012-Yellow-Mandarin-Ave-89081/home/29178219',
'https://www.redfin.com/NV/Las-Vegas/5921-Vegas-Dr-89108/home/29181770',
'https://www.redfin.com/NV/North-Las-Vegas/1103-Stoneypeak-Ave-89081/home/29236127',
'https://www.redfin.com/NV/North-Las-Vegas/4506-Minaret-Way-89031/home/29239506',
'https://www.redfin.com/NV/North-Las-Vegas/4028-Cricket-Ln-89032/home/29240817',
'https://www.redfin.com/NV/North-Las-Vegas/3733-Calumet-Farm-Cir-89031/home/29242138',
'https://www.redfin.com/NV/North-Las-Vegas/3709-Bordley-Way-89032/home/29244834',
'https://www.redfin.com/NV/North-Las-Vegas/4339-Sparta-Way-89032/home/29245206',
'https://www.redfin.com/NV/North-Las-Vegas/3516-Barrel-Race-Ct-89032/home/29245933',
'https://www.redfin.com/NV/North-Las-Vegas/4407-Red-Blanket-Rd-89032/home/29245989',
'https://www.redfin.com/NV/North-Las-Vegas/3716-Fountain-Ct-89032/home/29247433',
'https://www.redfin.com/NV/North-Las-Vegas/3820-Iverson-Ln-89032/home/29250323',
'https://www.redfin.com/NV/North-Las-Vegas/1881-W-Alexander-Rd-89032/unit-2149/home/29251690',
'https://www.redfin.com/NV/North-Las-Vegas/1516-Bluestar-Dr-89032/home/29252258',
'https://www.redfin.com/NV/North-Las-Vegas/3528-Beca-Faith-Dr-89032/home/29252430',
'https://www.redfin.com/NV/North-Las-Vegas/1532-Highfield-Ct-89032/home/29252706',
'https://www.redfin.com/NV/North-Las-Vegas/1324-Frost-Flower-Dr-89032/home/29253392',
'https://www.redfin.com/NV/North-Las-Vegas/3343-Outlook-Point-St-89032/home/29253559',
'https://www.redfin.com/NV/North-Las-Vegas/3301-Outlook-Point-St-89032/home/29253564',
'https://www.redfin.com/NV/North-Las-Vegas/3625-Rose-Canyon-Dr-89032/home/29254537',
'https://www.redfin.com/NV/North-Las-Vegas/3412-Mercury-St-89030/unit-I/home/29255374',
'https://www.redfin.com/NV/North-Las-Vegas/2200-Comet-Ave-89030/unit-C/home/29256236',
'https://www.redfin.com/NV/North-Las-Vegas/2120-Sun-Ave-89030/unit-C/home/29256298',
'https://www.redfin.com/NV/North-Las-Vegas/2112-Sun-Ave-89030/unit-F/home/29256306',
'https://www.redfin.com/NV/North-Las-Vegas/3470-Berg-St-89030/home/29256616',
'https://www.redfin.com/NV/North-Las-Vegas/2515-Bassler-St-89030/home/29258867',
'https://www.redfin.com/NV/North-Las-Vegas/2712-Holmes-St-89030/home/29259050',
'https://www.redfin.com/NV/North-Las-Vegas/2617-N-Bruce-St-89030/home/29261856',
'https://www.redfin.com/NV/Las-Vegas/4425-Cormorant-Ave-89115/home/29263467',
'https://www.redfin.com/NV/North-Las-Vegas/1871-Renada-Cir-89030/home/29263641',
'https://www.redfin.com/NV/North-Las-Vegas/3725-Citrus-Heights-Ave-89081/home/29264190',
'https://www.redfin.com/NV/North-Las-Vegas/3609-Inverness-Grove-Ave-89081/home/29264283',
'https://www.redfin.com/NV/North-Las-Vegas/3540-Turquoise-Waters-Ave-89081/home/29264707',
'https://www.redfin.com/NV/North-Las-Vegas/3612-Turquoise-Waters-Ave-89081/home/29264888',
'https://www.redfin.com/NV/North-Las-Vegas/2209-E-Piper-Ave-89030/home/29264893',
'https://www.redfin.com/NV/North-Las-Vegas/6745-Caporetto-Ln-89084/unit-102/home/29267106',
'https://www.redfin.com/NV/North-Las-Vegas/7070-Arcadia-Glen-Ct-89084/home/29267752',
'https://www.redfin.com/NV/North-Las-Vegas/316-Whitney-Breeze-Ave-89031/home/29268867',
'https://www.redfin.com/NV/North-Las-Vegas/216-Gemstone-Hill-Ave-89031/home/29269091',
'https://www.redfin.com/NV/North-Las-Vegas/2524-Spear-St-89030/home/29280897',
'https://www.redfin.com/NV/North-Las-Vegas/435-Dorchester-Bend-Ave-89032/unit-3/home/29280967',
'https://www.redfin.com/NV/Las-Vegas/2332-Bristol-View-Ct-89108/home/29305157',
'https://www.redfin.com/NV/Las-Vegas/4428-W-Lake-Mead-Blvd-89108/unit-202/home/29306179',
'https://www.redfin.com/NV/Las-Vegas/2117-Valley-Dr-89108/home/29306377',
'https://www.redfin.com/NV/Las-Vegas/1816-N-Decatur-Blvd-89108/unit-201/home/29307159',
'https://www.redfin.com/NV/Las-Vegas/4413-Coran-Ln-89108/home/29307372',
'https://www.redfin.com/NV/Las-Vegas/3008-Morningside-Ave-89106/home/29307757',
'https://www.redfin.com/NV/Las-Vegas/1812-Windchime-Dr-89106/home/29308825',
'https://www.redfin.com/NV/Las-Vegas/1816-Luning-Dr-89106/home/29310496',
'https://www.redfin.com/NV/Las-Vegas/2521-Perryville-Ave-89106/unit-104/home/29310634',
'https://www.redfin.com/NV/North-Las-Vegas/2037-Rose-St-89032/home/29311817',
'https://www.redfin.com/NV/Las-Vegas/875-Balzar-Ave-89106/home/29321865',
'https://www.redfin.com/NV/North-Las-Vegas/1912-Hoover-St-89030/home/29326589',
'https://www.redfin.com/NV/North-Las-Vegas/3208-Tabor-Ave-89030/home/29330401',
'https://www.redfin.com/NV/North-Las-Vegas/3309-Thomas-Ave-89030/home/29330529',
'https://www.redfin.com/NV/North-Las-Vegas/3701-Judson-Ave-89030/home/29330593',
'https://www.redfin.com/NV/North-Las-Vegas/3301-Judson-Ave-89030/home/29330642',
'https://www.redfin.com/NV/North-Las-Vegas/3615-Perliter-Ave-89030/home/29330934',
'https://www.redfin.com/NV/North-Las-Vegas/3420-E-Tonopah-Ave-89030/home/29331162',
'https://www.redfin.com/NV/Las-Vegas/1334-Smoke-Tree-Ave-89108/home/29338357',
'https://www.redfin.com/NV/Las-Vegas/1300-Smoke-Tree-Ave-89108/home/29338443',
'https://www.redfin.com/NV/Las-Vegas/3700-Briarglen-Ln-89108/home/29341997',
'https://www.redfin.com/NV/Las-Vegas/4248-Vornsand-Dr-89115/home/29352257',
'https://www.redfin.com/NV/Las-Vegas/4284-Vornsand-Dr-89115/home/29352271',
'https://www.redfin.com/NV/Las-Vegas/3730-Twinkle-Star-Dr-89115/home/29352775',
'https://www.redfin.com/NV/Las-Vegas/4270-Wendy-Ln-89115/home/29352909',
'https://www.redfin.com/NV/Las-Vegas/4022-Wendy-Ln-89115/home/29353796',
'https://www.redfin.com/NV/Las-Vegas/3871-Lancome-St-89115/home/29356974',
'https://www.redfin.com/NV/Las-Vegas/3757-Longfellow-St-89115/home/29357073',
'https://www.redfin.com/NV/North-Las-Vegas/5104-Harold-St-89081/home/29449056',
'https://www.redfin.com/NV/North-Las-Vegas/5948-Bellows-Beach-St-89081/home/29476800',
'https://www.redfin.com/NV/North-Las-Vegas/1010-Sunny-Acres-Ave-89081/home/29597120',
'https://www.redfin.com/NV/North-Las-Vegas/5062-Bayberry-Crest-St-89031/home/29598269',
'https://www.redfin.com/NV/North-Las-Vegas/3936-Fuselier-Dr-89032/home/29599396',
'https://www.redfin.com/NV/North-Las-Vegas/3829-Alpine-Track-Ave-89032/home/29781515',
'https://www.redfin.com/NV/North-Las-Vegas/5358-Ferrell-St-89031/home/29843745',
'https://www.redfin.com/NV/North-Las-Vegas/6662-Montezuma-Castle-Ln-89084/home/29851718',
'https://www.redfin.com/NV/North-Las-Vegas/1122-Granite-Ash-Ave-89081/home/29854357',
'https://www.redfin.com/NV/North-Las-Vegas/3411-Fort-Niagara-Ave-89032/home/29859302',
'https://www.redfin.com/NV/North-Las-Vegas/2430-Country-Orchard-St-89030/home/29859913',
'https://www.redfin.com/NV/North-Las-Vegas/2809-Albata-St-89030/home/29860046',
'https://www.redfin.com/NV/Las-Vegas/1631-Morse-Arberry-Ave-89106/home/29860968',
'https://www.redfin.com/NV/Las-Vegas/1704-Dwayne-Stedman-Ave-89106/home/29861006',
'https://www.redfin.com/NV/Las-Vegas/4730-E-Craig-Rd-89115/unit-1166/home/29862551',
'https://www.redfin.com/NV/North-Las-Vegas/2926-Walrus-Islands-Ct-89031/home/29886895',
'https://www.redfin.com/NV/North-Las-Vegas/5302-Big-Range-St-89031/home/29892203',
'https://www.redfin.com/NV/North-Las-Vegas/1102-Elliot-Park-Ave-89032/home/29894093',
'https://www.redfin.com/NV/North-Las-Vegas/1113-Christopher-View-Ave-89032/home/29894100',
'https://www.redfin.com/NV/North-Las-Vegas/5326-Kadena-Garden-Ct-89031/home/29919735',
'https://www.redfin.com/NV/Las-Vegas/6229-Eagle-Crossing-St-89130/home/29921118',
'https://www.redfin.com/NV/Las-Vegas/6493-Yellow-Bells-Ct-89131/home/29928568',
'https://www.redfin.com/NV/Las-Vegas/6493-Lantana-Falls-Ct-89131/home/29928645',
'https://www.redfin.com/NV/North-Las-Vegas/4716-Ferrell-St-89031/home/29932495',
'https://www.redfin.com/NV/North-Las-Vegas/6528-Bright-Morning-St-89084/home/29952095',
'https://www.redfin.com/NV/North-Las-Vegas/2112-Meadow-Green-Ave-89031/home/29954918',
'https://www.redfin.com/NV/North-Las-Vegas/5345-Brazelton-St-89081/home/30192840',
'https://www.redfin.com/NV/North-Las-Vegas/108-Elks-Peak-Ave-89084/home/33507294',
'https://www.redfin.com/NV/North-Las-Vegas/4137-Keasberry-Ave-89081/home/40232910',
'https://www.redfin.com/NV/Las-Vegas/4437-Dover-Straight-St-89115/unit-202/home/40234735',
'https://www.redfin.com/NV/Las-Vegas/4437-Dover-Straight-St-89115/unit-207/home/40234738',
'https://www.redfin.com/NV/North-Las-Vegas/3513-Storks-Bundle-Ln-89084/home/40534226',
'https://www.redfin.com/NV/North-Las-Vegas/5840-Montina-Vines-St-89081/home/52622144',
'https://www.redfin.com/NV/North-Las-Vegas/3808-Citrus-Heights-Ave-89081/home/65549955',
'https://www.redfin.com/NV/North-Las-Vegas/5741-Keystone-Crest-St-89081/home/92905186',
'https://www.redfin.com/NV/North-Las-Vegas/5741-Clear-Haven-Ln-89081/home/109620962',
'https://www.redfin.com/NV/North-Las-Vegas/5682-Colbert-St-89081/home/113139411',
'https://www.redfin.com/NV/North-Las-Vegas/6454-Cinnamon-Hazlenut-St-89084/home/143128831',
'https://www.redfin.com/NV/North-Las-Vegas/3796-Nerine-Pass-Way-89032/home/144626944',
'https://www.redfin.com/NV/North-Las-Vegas/5632-Akula-Bay-St-89081/home/145866395',
'https://www.redfin.com/NV/North-Las-Vegas/5853-Petrified-Tree-Ln-89081/home/145869556',
'https://www.redfin.com/NV/North-Las-Vegas/4556-Roaming-Vines-St-89031/home/145873671',
'https://www.redfin.com/NV/North-Las-Vegas/6239-Orions-Belt-Peak-St-89031/home/146417819',
'https://www.redfin.com/NV/North-Las-Vegas/1826-Yale-St-89030/home/147676348',
'https://www.redfin.com/NV/North-Las-Vegas/6508-Towerstone-St-89084/home/147875061',
'https://www.redfin.com/NV/North-Las-Vegas/6234-Orions-Belt-Peak-St-89031/home/147875671',
'https://www.redfin.com/NV/North-Las-Vegas/2416-Evening-Snow-Ct-89030/home/160497039',
'https://www.redfin.com/NV/North-Las-Vegas/3941-Aracari-Dr-89031/home/161751726',
'https://www.redfin.com/NV/North-Las-Vegas/5837-Delaney-Falls-St-89081/home/165730939',
'https://www.redfin.com/NV/North-Las-Vegas/537-Monarch-Ridge-Ct-89084/home/165736470',
'https://www.redfin.com/NV/North-Las-Vegas/4212-Juanita-May-Ave-89032/home/166108325',
'https://www.redfin.com/NV/North-Las-Vegas/3621-Zug-Bug-St-89032/home/166109615',
'https://www.redfin.com/NV/North-Las-Vegas/317-Stone-Fly-Rd-89032/home/166109624',
'https://www.redfin.com/NV/Las-Vegas/4481-Dover-Straight-St-89115/home/166121075',
'https://www.redfin.com/NV/North-Las-Vegas/4482-Brasada-Ranch-Ct-89031/home/166447829',
'https://www.redfin.com/NV/North-Las-Vegas/6938-Denio-Island-St-89084/home/166572294',
'https://www.redfin.com/NV/Las-Vegas/1285-Blankenship-Ave-89106/home/166583795',
'https://www.redfin.com/NV/North-Las-Vegas/7114-Red-Gulch-St-89084/home/167606567',
'https://www.redfin.com/NV/North-Las-Vegas/129-Mountain-Cedar-Ct-89031/home/167609307',
'https://www.redfin.com/NV/North-Las-Vegas/4531-Cradle-Cliff-St-89031/home/167654721',
'https://www.redfin.com/NV/North-Las-Vegas/6645-Evander-St-89086/home/169224926',
'https://www.redfin.com/NV/North-Las-Vegas/2316-Mystic-Wave-Rd-89031/home/170137348',
'https://www.redfin.com/NV/North-Las-Vegas/4355-Countryside-Glen-Ct-89084/home/170367594',
'https://www.redfin.com/NV/North-Las-Vegas/1006-Crystal-Grotto-Ave-89081/home/170368844',
'https://www.redfin.com/NV/North-Las-Vegas/4731-Agave-Cactus-St-89031/home/170372237',
'https://www.redfin.com/NV/North-Las-Vegas/6817-Toll-Mountain-St-89086/home/170968601',
'https://www.redfin.com/NV/North-Las-Vegas/2912-Sweeping-Palm-Rd-89086/home/170968949',
'https://www.redfin.com/NV/North-Las-Vegas/2908-Sweeping-Palm-Rd-89086/home/170968951',
'https://www.redfin.com/NV/North-Las-Vegas/2874-Roaming-Palmer-Rd-89086/home/170969037',
'https://www.redfin.com/NV/North-Las-Vegas/1201-Crystal-Grotto-Ave-89081/home/171287930',
'https://www.redfin.com/NV/North-Las-Vegas/6804-Surf-Green-Ct-89086/home/172603120',
'https://www.redfin.com/NV/North-Las-Vegas/6711-Palmer-Crest-St-89086/home/173453305',
'https://www.redfin.com/NV/North-Las-Vegas/4623-Observed-St-89031/home/173453834',
'https://www.redfin.com/NV/North-Las-Vegas/6734-Palmer-Estates-St-89086/home/174789327',
'https://www.redfin.com/NV/North-Las-Vegas/6722-Palmer-Estates-St-89086/home/174789330',
'https://www.redfin.com/NV/North-Las-Vegas/6959-Wildhorse-Creek-St-89086/home/175522419',
'https://www.redfin.com/NV/North-Las-Vegas/605-El-Gusto-Ave-89081/home/175522900',
'https://www.redfin.com/NV/North-Las-Vegas/2113-Dalewood-Ave-89086/home/176104742',
'https://www.redfin.com/NV/North-Las-Vegas/1062-Twincrest-Ave-89032/home/176107447',
'https://www.redfin.com/NV/North-Las-Vegas/2654-Fawn-Beige-Ct-89086/home/177325231',
'https://www.redfin.com/NV/North-Las-Vegas/6948-Crescent-Gold-St-89086/home/177325291',
'https://www.redfin.com/NV/North-Las-Vegas/6021-Torciano-St-89081/home/177334414',
'https://www.redfin.com/NV/North-Las-Vegas/7103-Stockton-Dunes-St-89084/home/179956239',
'https://www.redfin.com/NV/North-Las-Vegas/50-Gosia-Grove-Ave-89031/home/179957007',
'https://www.redfin.com/NV/North-Las-Vegas/46-Gosia-Grove-Ave-89031/home/179957008',
'https://www.redfin.com/NV/North-Las-Vegas/104-Barton-Bay-Ave-89031/home/179957017',
'https://www.redfin.com/NV/North-Las-Vegas/4517-Queens-Hill-St-89031/home/179978358',
'https://www.redfin.com/NV/North-Las-Vegas/4521-Queens-Hill-St-89031/home/179978359',
'https://www.redfin.com/NV/North-Las-Vegas/4526-Queens-Hill-St-89031/home/179978387',
'https://www.redfin.com/NV/North-Las-Vegas/1639-Oro-Grande-Dr-89032/home/179978478',
'https://www.redfin.com/NV/North-Las-Vegas/1635-Oro-Grande-Dr-89032/home/179978479',
'https://www.redfin.com/NV/North-Las-Vegas/1631-Oro-Grande-Dr-89032/home/179978480',
'https://www.redfin.com/NV/North-Las-Vegas/1590-Gold-Sunset-Ave-89032/home/179978615',
'https://www.redfin.com/NV/North-Las-Vegas/4144-Sandia-Star-Dr-89032/home/179978618',
'https://www.redfin.com/NV/North-Las-Vegas/4148-Sandia-Star-Dr-89032/home/179978619',
'https://www.redfin.com/NV/North-Las-Vegas/4156-Sandia-Star-Dr-89032/home/179978621',
'https://www.redfin.com/NV/North-Las-Vegas/6355-Selston-Dr-89081/home/181098995',
'https://www.redfin.com/NV/Las-Vegas/4932-Chidlaw-St-89115/unit-1/home/183991552',
'https://www.redfin.com/NV/Las-Vegas/4932-Chidlaw-St-89115/unit-3/home/183991559',
'https://www.redfin.com/NV/North-Las-Vegas/128-Junction-Peak-Ave-89031/home/185108853',
'https://www.redfin.com/NV/Las-Vegas/3710-Coleus-AVE-89108/unit-3/home/185911190',
'https://www.redfin.com/NV/North-Las-Vegas/6220-Demille-Ct-89081/home/28981990',
'https://www.redfin.com/NV/North-Las-Vegas/3012-Gnatcatcher-Ave-89084/home/29003013',
'https://www.redfin.com/NV/North-Las-Vegas/431-Yarnell-Dr-89031/home/29085276',
'https://www.redfin.com/NV/Las-Vegas/5904-Edrene-Ave-89108/home/29176233',
'https://www.redfin.com/NV/North-Las-Vegas/3520-Flower-Ave-89030/home/29330949',
'https://www.redfin.com/NV/North-Las-Vegas/1612-Logan-Valley-Ln-89081/home/29013124',
'https://www.redfin.com/NV/North-Las-Vegas/3932-Pepper-Thorn-Ave-89081/unit-2/home/29016303',
'https://www.redfin.com/NV/North-Las-Vegas/5239-Sugar-Maple-Ct-89031/home/29085148',
'https://www.redfin.com/NV/Las-Vegas/5609-Reiter-Ave-89108/home/29175388',
'https://www.redfin.com/NV/North-Las-Vegas/1304-E-Brooks-Ave-89030/home/29262132',
'https://www.redfin.com/NV/N-Las-Vegas/Address-Redacted-89031/home/29268765',
'https://www.redfin.com/NV/North-Las-Vegas/4350-Westbury-Square-St-89032/unit-1/home/29280978',
'https://www.redfin.com/NV/Las-Vegas/5344-Standing-Rock-Pl-89130/home/28967715',
'https://www.redfin.com/NV/Las-Vegas/5455-Cardinal-Ridge-Ct-89149/unit-202/home/28969903',
'https://www.redfin.com/NV/Las-Vegas/9421-Eagle-River-Ct-89166/home/28972613',
'https://www.redfin.com/NV/Las-Vegas/7917-Orchard-Port-Ave-89131/home/28977263',
'https://www.redfin.com/NV/Las-Vegas/8820-Saddle-Red-Ave-89143/home/28977819',
'https://www.redfin.com/NV/Las-Vegas/7104-Via-Campanile-Ave-89131/home/28979924',
'https://www.redfin.com/NV/Las-Vegas/9724-Red-Deer-St-89143/home/28983073',
'https://www.redfin.com/NV/Las-Vegas/8136-Nestled-Ct-89143/home/28983558',
'https://www.redfin.com/NV/Las-Vegas/8112-Loma-del-Ray-St-89131/home/28984267',
'https://www.redfin.com/NV/Las-Vegas/8036-Pasadera-St-89131/home/28984316',
'https://www.redfin.com/NV/Las-Vegas/5908-Audria-Falls-Ave-89131/home/28984945',
'https://www.redfin.com/NV/Las-Vegas/9546-Michelle-Falls-Ave-89149/home/29003683',
'https://www.redfin.com/NV/Las-Vegas/9305-Hampstead-Hills-Ave-89149/home/29004995',
'https://www.redfin.com/NV/Las-Vegas/9515-Havasu-Canyon-Ave-89166/home/29005497',
'https://www.redfin.com/NV/Las-Vegas/6476-Sandy-Copper-Ct-89131/home/29006117',
'https://www.redfin.com/NV/Las-Vegas/8065-Villa-Armando-St-89131/home/29008235',
'https://www.redfin.com/NV/Las-Vegas/8640-Painted-Horseshoe-St-89131/home/29009717',
'https://www.redfin.com/NV/Las-Vegas/9069-Adorable-Ave-89149/home/29011602',
'https://www.redfin.com/NV/Las-Vegas/7257-Royal-Melbourne-Dr-89131/home/29011630',
'https://www.redfin.com/NV/Las-Vegas/7404-Via-Fiorentino-St-89131/home/29015148',
'https://www.redfin.com/NV/Las-Vegas/8917-Monte-Oro-Dr-89131/home/29020290',
'https://www.redfin.com/NV/Las-Vegas/8917-Briar-Bay-Dr-89131/home/29020874',
'https://www.redfin.com/NV/Las-Vegas/8731-James-Raul-Ave-89143/home/29021009',
'https://www.redfin.com/NV/Las-Vegas/9136-Captivating-Ave-89149/home/29022297',
'https://www.redfin.com/NV/Las-Vegas/9141-Captivating-Ave-89149/home/29022318',
'https://www.redfin.com/NV/Las-Vegas/7913-Arden-Point-St-89149/home/29022805',
'https://www.redfin.com/NV/Las-Vegas/7852-Solid-Horn-Ct-89149/home/29023048',
'https://www.redfin.com/NV/Las-Vegas/9928-Kudo-Ct-89149/home/29023121',
'https://www.redfin.com/NV/Las-Vegas/6820-Sleepy-Pine-St-89130/home/29024467',
'https://www.redfin.com/NV/Las-Vegas/7580-Prancing-Pony-Ct-89131/home/29050606',
'https://www.redfin.com/NV/Las-Vegas/6632-Bluebell-Garden-St-89149/home/29051201',
'https://www.redfin.com/NV/Las-Vegas/8529-Caladium-Ct-89149/home/29051227',
'https://www.redfin.com/NV/Las-Vegas/6086-Denton-Ranch-Rd-89131/home/29051241',
'https://www.redfin.com/NV/Las-Vegas/6232-Tempting-Choice-Ave-89131/home/29060248',
'https://www.redfin.com/NV/Las-Vegas/8137-Satin-Carnation-Ln-89166/home/29060435',
'https://www.redfin.com/NV/Las-Vegas/6125-Tarrant-Ranch-Rd-89131/home/29060816',
'https://www.redfin.com/NV/Las-Vegas/6955-N-Durango-Dr-89149/unit-1047/home/29061019',
'https://www.redfin.com/NV/Las-Vegas/6955-N-Durango-Dr-89149/unit-1075/home/29061034',
'https://www.redfin.com/NV/Las-Vegas/3975-N-Hualapai-Way-89129/unit-256/home/29061259',
'https://www.redfin.com/NV/Las-Vegas/3975-N-Hualapai-Way-89129/unit-228/home/29061315',
'https://www.redfin.com/NV/Las-Vegas/8709-Grazing-Hill-Ct-89143/home/29087101',
'https://www.redfin.com/NV/Las-Vegas/8937-Iron-Hitch-Ave-89143/home/29087218',
'https://www.redfin.com/NV/Las-Vegas/9775-Homestead-Rd-89143/home/29087373',
'https://www.redfin.com/NV/Las-Vegas/8921-Tumblewood-Ave-89143/home/29087512',
'https://www.redfin.com/NV/Las-Vegas/8729-Burning-Hide-Ave-89143/home/29087661',
'https://www.redfin.com/NV/Las-Vegas/8424-Wild-Diamond-Ave-89143/home/29087946',
'https://www.redfin.com/NV/Las-Vegas/7820-Thorne-Pine-Ave-89131/home/29090088',
'https://www.redfin.com/NV/Las-Vegas/7809-Amulet-St-89131/home/29090263',
'https://www.redfin.com/NV/Las-Vegas/7905-Elk-Stone-Ave-89131/home/29090763',
'https://www.redfin.com/NV/Las-Vegas/7921-Lovely-Pine-Pl-89143/home/29091078',
'https://www.redfin.com/NV/Las-Vegas/8112-Boxberry-Ave-89131/home/29091590',
'https://www.redfin.com/NV/Las-Vegas/7804-Purple-Mountain-Ave-89131/home/29091702',
'https://www.redfin.com/NV/Las-Vegas/8205-Antler-Pines-Ct-89149/home/29091914',
'https://www.redfin.com/NV/Las-Vegas/6812-Tartarus-St-89131/home/29092130',
'https://www.redfin.com/NV/Las-Vegas/6924-Bienville-St-89131/home/29092930',
'https://www.redfin.com/NV/Las-Vegas/7213-Grand-Palace-Ave-89130/home/29095278',
'https://www.redfin.com/NV/Las-Vegas/5900-Taj-Mahal-Dr-89130/home/29095508',
'https://www.redfin.com/NV/Las-Vegas/5924-Kane-Holly-St-89130/home/29095513',
'https://www.redfin.com/NV/Las-Vegas/8040-Edward-Baher-Ave-89149/home/29095775',
'https://www.redfin.com/NV/Las-Vegas/9662-Azure-Dr-89149/home/29096996',
'https://www.redfin.com/NV/Las-Vegas/9665-W-Regena-Ave-89149/home/29096999',
'https://www.redfin.com/NV/Las-Vegas/5245-N-Riley-St-89149/home/29097445',
'https://www.redfin.com/NV/Las-Vegas/9045-W-Rosada-Way-89149/home/29097602',
'https://www.redfin.com/NV/Las-Vegas/9085-W-Rosada-Way-89149/home/29097603',
'https://www.redfin.com/NV/Las-Vegas/8324-Green-Clover-Ave-89149/home/29098000',
'https://www.redfin.com/NV/Las-Vegas/8221-Green-Clover-Ave-89149/home/29098049',
'https://www.redfin.com/NV/Las-Vegas/8101-Kimwood-Ave-89149/home/29098193',
'https://www.redfin.com/NV/Las-Vegas/7620-Sudan-Ct-89149/home/29098822',
'https://www.redfin.com/NV/Las-Vegas/5124-N-Valadez-St-89149/home/29099131',
'https://www.redfin.com/NV/Las-Vegas/7448-Desert-Flame-Ct-89149/home/29099277',
'https://www.redfin.com/NV/Las-Vegas/6812-Bottle-Sage-Ave-89130/home/29099692',
'https://www.redfin.com/NV/Las-Vegas/6909-Squaw-Mountain-Dr-89130/unit-202/home/29099788',
'https://www.redfin.com/NV/Las-Vegas/6928-White-Lakes-Ave-89130/home/29099821',
'https://www.redfin.com/NV/Las-Vegas/6717-Snake-River-Ave-89130/home/29099890',
'https://www.redfin.com/NV/Las-Vegas/4920-Black-Bear-Rd-89149/unit-202/home/29100384',
'https://www.redfin.com/NV/Las-Vegas/4830-Grey-Wolf-Ln-89149/unit-203/home/29100732',
'https://www.redfin.com/NV/Las-Vegas/4816-Storm-Mountain-St-89130/home/29101019',
'https://www.redfin.com/NV/Las-Vegas/5520-Jasper-Butte-St-89130/home/29101382',
'https://www.redfin.com/NV/Las-Vegas/6031-Ruffian-Rd-89149/home/29104232',
'https://www.redfin.com/NV/Las-Vegas/6421-Faith-Peak-Dr-89108/home/29117156',
'https://www.redfin.com/NV/Las-Vegas/6616-Ouida-Way-89108/home/29117824',
'https://www.redfin.com/NV/Las-Vegas/4101-Browndeer-Cir-89129/home/29119347',
'https://www.redfin.com/NV/Las-Vegas/4028-Compass-Rose-Way-89108/home/29119619',
'https://www.redfin.com/NV/Las-Vegas/7704-Wedlock-Ln-89129/home/29120301',
'https://www.redfin.com/NV/Las-Vegas/7717-Wedlock-Ln-89129/home/29120310',
'https://www.redfin.com/NV/Las-Vegas/4355-Kevin-Way-89129/home/29120783',
'https://www.redfin.com/NV/Las-Vegas/4001-Antique-Sterling-Ct-89129/home/29120810',
'https://www.redfin.com/NV/Las-Vegas/4785-N-Grand-Canyon-Dr-89129/home/29121009',
'https://www.redfin.com/NV/Las-Vegas/8105-Defiance-Ave-89129/home/29126336',
'https://www.redfin.com/NV/Las-Vegas/3888-Quadrel-St-89129/home/29127580',
'https://www.redfin.com/NV/Las-Vegas/3912-Crema-Ct-89129/home/29128157',
'https://www.redfin.com/NV/Las-Vegas/3804-Bessemer-Ct-89129/home/29128647',
'https://www.redfin.com/NV/Las-Vegas/6557-Canyon-Cove-Way-89108/home/29130773',
'https://www.redfin.com/NV/Las-Vegas/6317-W-Alexander-Rd-89108/home/29131260',
'https://www.redfin.com/NV/Las-Vegas/8250-N-Grand-Canyon-Dr-89166/unit-2184/home/29179937',
'https://www.redfin.com/NV/Las-Vegas/6451-Iguassu-Falls-Rd-89149/home/29180079',
'https://www.redfin.com/NV/Las-Vegas/8318-Alameda-Falls-Ave-89131/home/29271892',
'https://www.redfin.com/NV/Las-Vegas/9336-Cranesbill-Ct-89149/home/29273782',
'https://www.redfin.com/NV/Las-Vegas/9252-Lopseed-Ct-89149/home/29273839',
'https://www.redfin.com/NV/Las-Vegas/6805-Mansion-Oaks-St-89149/home/29274068',
'https://www.redfin.com/NV/Las-Vegas/8448-Lucero-Ave-89149/home/29275197',
'https://www.redfin.com/NV/Las-Vegas/6868-Sky-Pointe-Dr-89131/unit-2083/home/29275396',
'https://www.redfin.com/NV/Las-Vegas/10769-Tuckermans-Ave-89129/home/29276541',
'https://www.redfin.com/NV/Las-Vegas/10438-Gold-Shadow-Ave-89129/home/29277178',
'https://www.redfin.com/NV/Las-Vegas/7971-Lookout-Rock-Cir-89129/home/29479245',
'https://www.redfin.com/NV/Las-Vegas/9535-New-Harbor-Ave-89149/home/29598062',
'https://www.redfin.com/NV/Las-Vegas/8562-Golden-Idol-Ct-89131/home/29598146',
'https://www.redfin.com/NV/Las-Vegas/10715-Iona-Island-Ave-89166/home/29759373',
'https://www.redfin.com/NV/Las-Vegas/7627-Anatolian-St-89166/home/29759399',
'https://www.redfin.com/NV/Las-Vegas/10148-Ocicat-Ave-89166/home/29759419',
'https://www.redfin.com/NV/Las-Vegas/10516-Nantucket-Ridge-Ave-89166/home/29759684',
'https://www.redfin.com/NV/Las-Vegas/6631-Brooklyn-Heights-St-89166/home/29760650',
'https://www.redfin.com/NV/Las-Vegas/10408-Bays-Mountain-Ave-89166/home/29774223',
'https://www.redfin.com/NV/Las-Vegas/8921-Teetering-Rock-Ave-89143/home/29843595',
'https://www.redfin.com/NV/Las-Vegas/7804-Glowing-Peak-St-89131/home/29844676',
'https://www.redfin.com/NV/Las-Vegas/8304-Spanish-Meadows-Ave-89131/home/29892038',
'https://www.redfin.com/NV/Las-Vegas/4435-Peaceful-Heights-Ln-89129/home/29893836',
'https://www.redfin.com/NV/Las-Vegas/5049-Indigo-Gorge-Ave-89131/home/29916708',
'https://www.redfin.com/NV/Las-Vegas/7805-Brent-Leaf-Ave-89131/home/29927599',
'https://www.redfin.com/NV/Las-Vegas/3855-Wiggins-Bay-St-89129/unit-104/home/29930047',
'https://www.redfin.com/NV/Las-Vegas/10125-Cedar-Key-Ave-89129/unit-101/home/29930421',
'https://www.redfin.com/NV/Las-Vegas/9329-Euphoria-Rose-Ave-89166/home/30040711',
'https://www.redfin.com/NV/Las-Vegas/6892-Deer-Isle-St-89149/home/30377787',
'https://www.redfin.com/NV/Las-Vegas/10132-Ocicat-Ave-89166/home/30469451',
'https://www.redfin.com/NV/Las-Vegas/7324-Arlington-Garden-St-89166/home/30469481',
'https://www.redfin.com/NV/Las-Vegas/10529-Bandera-Mountain-Ln-89166/home/40297544',
'https://www.redfin.com/NV/Las-Vegas/7149-Flagstaff-Ranch-St-89166/home/40348542',
'https://www.redfin.com/NV/Las-Vegas/7506-Derby-Wharf-Ct-89166/home/49871406',
'https://www.redfin.com/NV/Las-Vegas/6434-Affermato-St-89131/home/51677901',
'https://www.redfin.com/NV/Las-Vegas/3948-Deluge-Dr-89129/home/52624661',
'https://www.redfin.com/NV/Las-Vegas/7718-Albright-Peak-Dr-89166/home/52625313',
'https://www.redfin.com/NV/Las-Vegas/7327-Brisbane-Hills-St-89166/home/52625334',
'https://www.redfin.com/NV/Las-Vegas/10313-Jolly-Hill-Ave-89166/home/52625380',
'https://www.redfin.com/NV/Las-Vegas/10300-Gibson-Isle-Dr-89166/home/52629297',
'https://www.redfin.com/NV/Las-Vegas/7055-Thistle-Ridge-St-89166/home/56390362',
'https://www.redfin.com/NV/Las-Vegas/5961-Golden-Arowana-Way-89149/home/58810308',
'https://www.redfin.com/NV/Las-Vegas/10744-Merrimack-Ave-89166/home/65555835',
'https://www.redfin.com/NV/Las-Vegas/10725-Crosley-Field-Ave-89166/home/65557121',
'https://www.redfin.com/NV/Las-Vegas/9563-W-Lone-Mountain-Rd-89129/home/65557470',
'https://www.redfin.com/NV/Las-Vegas/8153-Smoking-Jacket-Pl-89166/home/92906248',
'https://www.redfin.com/NV/Las-Vegas/9762-Champagne-Bliss-Ave-89149/home/103972196',
'https://www.redfin.com/NV/Las-Vegas/10804-Leatherstocking-Ave-89166/home/109621118',
'https://www.redfin.com/NV/Las-Vegas/10131-Mayflower-Bay-Ave-89166/home/109621865',
'https://www.redfin.com/NV/Las-Vegas/10439-Prairie-Mountain-Ave-89166/home/109860997',
'https://www.redfin.com/NV/Las-Vegas/7564-Chimney-Point-Ct-89166/home/109862484',
'https://www.redfin.com/NV/Las-Vegas/10757-Wrigley-Field-Ave-89166/home/109864889',
'https://www.redfin.com/NV/Las-Vegas/8391-Aspen-Skye-St-89166/home/109872265',
'https://www.redfin.com/NV/Las-Vegas/7852-Blue-Lake-Peak-St-89166/home/113514096',
'https://www.redfin.com/NV/Las-Vegas/5570-Moore-Cove-Ave-89131/home/145872365',
'https://www.redfin.com/NV/Las-Vegas/7839-Castle-Rock-Peak-St-89166/home/147381249',
'https://www.redfin.com/NV/Las-Vegas/7339-Winesburg-St-89166/home/147874048',
'https://www.redfin.com/NV/Las-Vegas/8302-Skye-Territory-St-89166/home/165776376',
'https://www.redfin.com/NV/Las-Vegas/10146-Skye-Run-Ave-89166/home/165776590',
'https://www.redfin.com/NV/Las-Vegas/6120-N-Monte-Cristo-Way-89130/home/166086307',
'https://www.redfin.com/NV/Las-Vegas/8064-Big-Skye-Ranch-St-89166/home/166088788',
'https://www.redfin.com/NV/Las-Vegas/4377-Velvet-Ridge-Ct-89129/home/166531094',
'https://www.redfin.com/NV/Las-Vegas/10267-Timberline-Valley-Ave-89166/home/166576290',
'https://www.redfin.com/NV/Las-Vegas/10673-Scorched-Skye-Ave-89166/home/167628890',
'https://www.redfin.com/NV/Las-Vegas/8439-Carbon-Canyon-Ct-89129/home/167639222',
'https://www.redfin.com/NV/Las-Vegas/10697-Sariah-Skye-Ave-89166/home/169227559',
'https://www.redfin.com/NV/Las-Vegas/8090-Skye-Dragon-St-89166/home/169227672',
'https://www.redfin.com/NV/Las-Vegas/8048-Skye-Dragon-St-89166/home/169227679',
'https://www.redfin.com/NV/Las-Vegas/10484-Skye-Knoll-Ave-89166/home/169228004',
'https://www.redfin.com/NV/Las-Vegas/8323-Lagunilla-Ave-89149/home/170969787',
'https://www.redfin.com/NV/Las-Vegas/9975-Skye-Gazer-Ave-89166/home/171301529',
'https://www.redfin.com/NV/Las-Vegas/Ruffian-Rd-89149/home/174980410',
'https://www.redfin.com/NV/Las-Vegas/10125-Tree-Blossom-Ave-89166/home/176105333',
'https://www.redfin.com/NV/Las-Vegas/8657-Hartmont-St-89166/home/176105384',
'https://www.redfin.com/NV/Las-Vegas/9601-Jadeite-Ln-89143/home/176643630',
'https://www.redfin.com/NV/Las-Vegas/9827-Mojave-Mountain-St-89166/home/176643796',
'https://www.redfin.com/NV/Las-Vegas/9006-Laurel-Creek-St-89166/home/177117836',
'https://www.redfin.com/NV/Las-Vegas/9001-Pinyon-Bluff-St-89166/home/177117840',
'https://www.redfin.com/NV/Las-Vegas/8989-Pinyon-Bluff-St-89166/home/177117842',
'https://www.redfin.com/NV/Las-Vegas/8935-Windgate-Ranch-St-89166/home/177117950',
'https://www.redfin.com/NV/Las-Vegas/7358-Hadnot-St-89166/home/177118470',
'https://www.redfin.com/NV/Las-Vegas/6546-Castalnova-Bend-Dr-89166/home/177118928',
'https://www.redfin.com/NV/Las-Vegas/6556-Sandero-Springs-St-89166/home/177118988',
'https://www.redfin.com/NV/Las-Vegas/6485-Castalnova-Bend-Dr-89166/home/177119199',
'https://www.redfin.com/NV/Las-Vegas/6515-Corsari-Ridge-St-89166/home/177119246',
'https://www.redfin.com/NV/Las-Vegas/9392-Lunar-Ray-St-89143/home/177326635',
'https://www.redfin.com/NV/Las-Vegas/5120-Olive-Leaf-Ct-89130/home/179963904',
'https://www.redfin.com/NV/Las-Vegas/10596-Bowmore-Ave-89166/home/179964464',
'https://www.redfin.com/NV/Las-Vegas/10375-Sadie-Blake-Ave-89166/home/179980888',
'https://www.redfin.com/NV/Las-Vegas/10391-Thurso-Ave-89166/home/179980934',
'https://www.redfin.com/NV/Las-Vegas/10399-Thurso-Ave-89166/home/179980935',
'https://www.redfin.com/NV/Las-Vegas/10416-Andele-Ave-89166/home/179980938',
'https://www.redfin.com/NV/Las-Vegas/10948-Dixie-Johnson-Ave-89166/home/179981077',
'https://www.redfin.com/NV/Las-Vegas/10854-Trail-Mark-Ave-89166/home/179981328',
'https://www.redfin.com/NV/Las-Vegas/10812-Gentle-Crest-Ave-89166/home/179981381',
'https://www.redfin.com/NV/Las-Vegas/6521-W-Azure-Dr-89130/home/181106130',
'https://www.redfin.com/NV/Las-Vegas/10336-Ansley-Bay-Ave-89166/home/181146167',
'https://www.redfin.com/NV/Las-Vegas/10344-Ansley-Bay-Ave-89166/home/181146168',
'https://www.redfin.com/NV/Las-Vegas/10343-Ansley-Bay-Ave-89166/home/181146169',
'https://www.redfin.com/NV/Las-Vegas/10335-Ansley-Bay-Ave-89166/home/181146170',
'https://www.redfin.com/NV/Las-Vegas/10327-Ansley-Bay-Ave-89166/home/181146171',
'https://www.redfin.com/NV/Las-Vegas/5473-Hickory-Tree-St-89149/unit-14/home/185224192',
'https://www.redfin.com/NV/Las-Vegas/5425-Hickory-Tree-St-89149/unit-16/home/185268980',
'https://www.redfin.com/NV/Las-Vegas/10965-Parsley-AVE-89166/unit-75/home/185735772',
'https://www.redfin.com/NV/Las-Vegas/10974-Parsley-AVE-89166/unit-82/home/185764041',
'https://www.redfin.com/NV/Las-Vegas/10760-Hickam-Ave-89129/unit-68/home/185839508',
'https://www.redfin.com/NV/Las-Vegas/10952-Kingston-Fields-AVE-Unknown/unit-237/home/185972384',
'https://www.redfin.com/NV/Las-Vegas/10960-Kingston-Fields-AVE-89166/unit-239/home/186067031',
'https://www.redfin.com/NV/Las-Vegas/9568-Jadeite-Ln-89143/unit-159/home/186218858',
'https://www.redfin.com/NV/Las-Vegas/10071-Chrysolite-Pl-89143/unit-218/home/186218860',
'https://www.redfin.com/NV/Las-Vegas/10039-Chrysolite-Pl-89143/unit-214/home/186218883',
'https://www.redfin.com/NV/Las-Vegas/4249-Lily-Troter-St-89084/unit-34/home/186430476',
'https://www.redfin.com/NV/Las-Vegas/9040-Proud-Ct-89149/home/29022244',
'https://www.redfin.com/NV/Las-Vegas/9124-Goose-Lake-Way-89149/unit-101/home/29023883',
'https://www.redfin.com/NV/Las-Vegas/7685-N-Rainbow-Blvd-89131/home/29088978',
'https://www.redfin.com/NV/Las-Vegas/8305-Alder-Meadows-Ave-89131/home/29089916',
'https://www.redfin.com/NV/Las-Vegas/4005-Compass-Rose-Way-89108/home/29119665',
'https://www.redfin.com/NV/Las-Vegas/10722-Monaco-Beach-Ave-89166/home/29759630',
'https://www.redfin.com/NV/Las-Vegas/5021-River-Splash-Ave-89131/home/29908637',
'https://www.redfin.com/NV/Las-Vegas/Address-Redacted-89166/home/40548880',
'https://www.redfin.com/NV/Las-Vegas/6885-W-Red-Coach-Ave-89108/home/165797110',
'https://www.redfin.com/NV/Las-Vegas/9568-Jadeite-Ln-89143/home/176643637',
'https://www.redfin.com/NV/Las-Vegas/9552-Jadeite-Ln-89143/home/176643639',
'https://www.redfin.com/NV/Las-Vegas/10947-Cheshire-Woods-Ave-89166/home/177118769',
'https://www.redfin.com/NV/Las-Vegas/10027-Carmine-Clay-Dr-89143/home/179962331',
'https://www.redfin.com/NV/Las-Vegas/10100-Carmine-Clay-Dr-89143/home/179962355',
'https://www.redfin.com/NV/Las-Vegas/10076-Carmine-Clay-Dr-89143/home/179962359',
'https://www.redfin.com/NV/Las-Vegas/10072-Carmine-Clay-Dr-89143/home/179962360',
'https://www.redfin.com/NV/Las-Vegas/10071-Chrysolite-Pl-89143/home/179962373',
'https://www.redfin.com/NV/Las-Vegas/11730-Stonewall-Springs-Ave-89138/home/28985995',
'https://www.redfin.com/NV/Las-Vegas/3504-Desert-Cliff-St-89129/unit-103/home/29006488',
'https://www.redfin.com/NV/Las-Vegas/10524-Danielson-Ave-89129/home/29024389',
'https://www.redfin.com/NV/Las-Vegas/11963-Prada-Verde-Dr-89138/home/29025419',
'https://www.redfin.com/NV/Las-Vegas/620-Apricot-Rose-Pl-89138/home/29025667',
'https://www.redfin.com/NV/Las-Vegas/605-Tandoori-Ln-89138/home/29051442',
'https://www.redfin.com/NV/Las-Vegas/825-Canterra-St-89138/unit-2059/home/29051569',
'https://www.redfin.com/NV/Las-Vegas/2451-N-Rainbow-Blvd-89108/unit-2128/home/29062127',
'https://www.redfin.com/NV/Las-Vegas/10604-Paradise-Point-Dr-89134/home/29105002',
'https://www.redfin.com/NV/Las-Vegas/10212-Pastel-Sky-Ave-89129/home/29105063',
'https://www.redfin.com/NV/Las-Vegas/10304-Back-Plains-Dr-89134/home/29105983',
'https://www.redfin.com/NV/Las-Vegas/10421-Villa-Ridge-Dr-89134/home/29106077',
'https://www.redfin.com/NV/Las-Vegas/2729-Herons-Creek-Dr-89134/home/29106205',
'https://www.redfin.com/NV/Las-Vegas/10809-Clarion-Ln-89134/home/29107551',
'https://www.redfin.com/NV/Las-Vegas/9404-Warm-Waters-Ave-89129/home/29121858',
'https://www.redfin.com/NV/Las-Vegas/3705-Silver-Brook-St-89129/home/29125232',
'https://www.redfin.com/NV/Las-Vegas/3620-Beacon-Point-St-89129/home/29125875',
'https://www.redfin.com/NV/Las-Vegas/8705-Texas-Ranger-Ave-89129/home/29126208',
'https://www.redfin.com/NV/Las-Vegas/6840-Elm-Creek-Dr-89108/unit-204/home/29130460',
'https://www.redfin.com/NV/Las-Vegas/3240-Serene-Dr-89108/home/29131284',
'https://www.redfin.com/NV/Las-Vegas/3637-Solitude-Rd-89108/home/29131469',
'https://www.redfin.com/NV/Las-Vegas/8112-Lake-Hills-Dr-89128/home/29142220',
'https://www.redfin.com/NV/Las-Vegas/3125-N-Buffalo-Dr-89128/unit-2083/home/29145136',
'https://www.redfin.com/NV/Las-Vegas/9013-Faircrest-Dr-89134/home/29146967',
'https://www.redfin.com/NV/Las-Vegas/8913-Desert-Mound-Dr-89134/home/29148240',
'https://www.redfin.com/NV/Las-Vegas/3101-Goodnews-Ct-89134/home/29148821',
'https://www.redfin.com/NV/Las-Vegas/9413-Villa-Ridge-Dr-89134/home/29152349',
'https://www.redfin.com/NV/Las-Vegas/1908-Tierra-Vista-Dr-89128/unit-206/home/29165343',
'https://www.redfin.com/NV/Las-Vegas/6249-Rassler-Ave-89107/home/29196305',
'https://www.redfin.com/NV/Las-Vegas/3151-Soaring-Gulls-Dr-89128/unit-2040/home/29242595',
'https://www.redfin.com/NV/Las-Vegas/3151-Soaring-Gulls-Dr-89128/unit-1194/home/29315121',
'https://www.redfin.com/NV/Las-Vegas/3151-Soaring-Gulls-Dr-89128/unit-1201/home/29315162',
'https://www.redfin.com/NV/Las-Vegas/11441-Allerton-Park-Dr-89135/unit-413/home/29488322',
'https://www.redfin.com/NV/Las-Vegas/1800-Hill-Canyon-Ln-89144/home/29846765',
'https://www.redfin.com/NV/Las-Vegas/3031-Raywood-Ash-Dr-89138/home/165784182',
'https://www.redfin.com/NV/Las-Vegas/748-Banff-Grounds-St-89138/home/170137802',
'https://www.redfin.com/NV/Las-Vegas/11553-Cedar-Shore-Ave-89138/home/173620671',
'https://www.redfin.com/NV/Las-Vegas/424-Cascade-Heights-Ave-89138/home/176106472',
'https://www.redfin.com/NV/Las-Vegas/11489-Vibrant-Heights-DR-89135/unit-33/home/178973247',
'https://www.redfin.com/NV/Las-Vegas/11461-Vibrant-Heights-DR-89135/unit-26/home/179105277',
'https://www.redfin.com/NV/Las-Vegas/11477-Vibrant-Heights-DR-89135/unit-30/home/179105536',
'https://www.redfin.com/NV/Las-Vegas/11620-Desert-Hollow-Ave-89138/home/179981632',
'https://www.redfin.com/NV/Las-Vegas/11493-Vibrant-Heights-DR-89135/unit-34/home/178973290',
'https://www.redfin.com/NV/Las-Vegas/11457-Vibrant-Heights-DR-89135/unit-25/home/179105251',
'https://www.redfin.com/NV/Las-Vegas/11604-Desert-Hollow-Ave-89138/home/179981627',
'https://www.redfin.com/NV/Las-Vegas/11615-Desert-Hollow-Ave-89138/home/179981730',
'https://www.redfin.com/NV/Las-Vegas/2267-High-Perch-St-89138/home/181147097',
'https://www.redfin.com/NV/Las-Vegas/11731-Hatchling-Ave-89138/home/181147154',
'https://www.redfin.com/NV/Las-Vegas/11832-Barcinas-Ln-89138/home/29025442',
'https://www.redfin.com/NV/Las-Vegas/11328-Asilo-Bianco-Ave-89138/home/29009926',
'https://www.redfin.com/NV/Las-Vegas/2130-Sealion-Dr-89128/unit-207/home/29026310',
'https://www.redfin.com/NV/Las-Vegas/3141-Ravenshoe-Dr-89134/home/29105138',
'https://www.redfin.com/NV/Las-Vegas/2125-Hallston-St-89134/home/29108322',
'https://www.redfin.com/NV/Las-Vegas/10317-Bent-Brook-Pl-89134/home/29109289',
'https://www.redfin.com/NV/Las-Vegas/1600-Rappeling-Ct-89144/home/29110085',
'https://www.redfin.com/NV/Las-Vegas/2549-Monarch-Bay-Dr-89128/home/29147165',
'https://www.redfin.com/NV/Las-Vegas/9036-Litchfield-Ave-89134/home/29147931',
'https://www.redfin.com/NV/Las-Vegas/2016-Waterbury-Ln-89134/home/29159654',
'https://www.redfin.com/NV/Las-Vegas/7929-Sierra-Largo-Dr-89128/home/29165299',
'https://www.redfin.com/NV/Las-Vegas/6609-Tampa-Ct-89108/home/29169727',
'https://www.redfin.com/NV/Las-Vegas/5817-Apple-Valley-Ln-89108/home/29186432',
'https://www.redfin.com/NV/Las-Vegas/7960-Terrace-Rock-Way-89128/unit-202/home/29205489',
'https://www.redfin.com/NV/Las-Vegas/133-Lisett-St-89138/home/176106680',
'https://www.redfin.com/NV/Las-Vegas/145-Lisett-St-89138/home/176106681',
'https://www.redfin.com/NV/Las-Vegas/506-Mahogany-Ridge-St-89138/home/173620651',
'https://www.redfin.com/NV/Las-Vegas/11537-Cedar-Shore-Ave-89138/home/173620669',
'https://www.redfin.com/NV/Las-Vegas/493-Highview-Ridge-Ave-89138/home/176105618',
'https://www.redfin.com/NV/Las-Vegas/12172-Harris-Summit-Ave-89138/home/176105984',
'https://www.redfin.com/NV/Las-Vegas/11688-Emerald-Lake-Ave-89138/home/177301808',

]  # Add more URLs as required
combined_data = [combine_data_from_url(url) for url in urls]

# Export data to JSON
with open('data.json', 'w') as f:
    json.dump(combined_data, f, indent=4)

# Update csv_columns to include new fields
csv_columns = [
    "URL", 
    "Walk Score (out of 100)", 
    "Transit Score (out of 100)", 
    "Bike Score (out of 100)",
    "Elementary School Name", 
    "Elementary School Score (out of 10)",
    "Middle School Name", 
    "Middle School Score (out of 10)",
    "High School Name", 
    "High School Score (out of 10)",
    "Address",
    "Price",
    "Beds",
    "Baths",
    "Sq Ft",
    "Est. Monthly Payment",
    "Time on Redfin",
    "Year Built",
    "Property Type",
    "Style",
    "HOA Dues",
    "Lot Size",
    "Garage Spaces",
    "Has Pool",
    "Floor Type",
    "Heat Type",
    "Cool Type",
    "Tax Amount",
    "Has Patio",
   
]


csv_file = "data.csv"
try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in combined_data:
            writer.writerow(data)
except IOError:
    print("I/O error")

C:\Users\binhd\AppData\Local\Temp\ipykernel_39964\936474545.py:104: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  exterior_features_div = soup.find('div', {'class': 'amenity-group'}, text=re.compile('Exterior Features'))
C:\Users\binhd\AppData\Local\Temp\ipykernel_39964\936474545.py:100: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  element = parent.find(text=re.compile(text))


Testing script


In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def parse_factors(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    factors = {}

    headings = soup.find_all('h3', class_='ListItem__heading font-body-base-bold color-text-primary')
    for heading in headings:
        text_content = heading.text
        if ' - ' in text_content:
            factor, value = text_content.split(' - ', 1)
            factors[factor] = value

    return factors

def save_to_csv(factors_list):
    df = pd.DataFrame(factors_list)
    df.to_csv('factors.csv', index=False)

def main():
    urls = [
        'https://www.redfin.com/NV/Las-Vegas/1913-E-Saint-Louis-Ave-89104/home/29454745',
'https://www.redfin.com/NV/Las-Vegas/3506-Westleigh-Ave-89102/home/29458768',
'https://www.redfin.com/NV/Las-Vegas/1355-Douglas-Dr-89102/home/29458770',
'https://www.redfin.com/NV/Las-Vegas/2101-Plaza-del-Cerro-89102/home/29459184',
'https://www.redfin.com/NV/Las-Vegas/2724-Miraflores-Ave-89102/home/29460615',
'https://www.redfin.com/NV/Las-Vegas/4557-Sirius-Ave-89102/home/29462921',
'https://www.redfin.com/NV/Las-Vegas/2824-Alcoa-Ave-89102/home/29464997',
'https://www.redfin.com/NV/Las-Vegas/2616-Yardley-St-89102/home/29465095',
        # ... add more URLs as needed
    ]

    factors_list = []
    for url in urls:
        factors = parse_factors(url)
        factors['URL'] = url  # Add the URL to the factors dictionary
        factors_list.append(factors)

    save_to_csv(factors_list)

if __name__ == '__main__':
    main()
